# Setup
Early phases of this project included coded, including functions with which to scrape and parse Wikipedia pages and full Wikipedia file dumps, that did not end up relevant to my final product so I have not included that here. My source data did come exclusively from Wikipedia, however. In my Python files I have included light documentation for the functions I used to sourced them, although a few of the earliest may not be included. Below, I've pieced together my workflow pieced as best I can, and hopefully in a way that can be reasonably understood by anyone reading it.

# Imports

In [ ]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time, os
from fake_useragent import UserAgent
import datetime
from tqdm import tqdm
import pickle
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import unidecode
from tensorflow import keras
import Functions as food

pd.set_option("display.max_rows", 300)

# Building a df with cuisine Wikipedia page data for every country + territories
Some of what I pulled into this df wasn't relevant in the final product, or at least isn't yet.

## First getting cuisine page links by continent/region
There was a lot of manual work, much of which could be refined, in the below.

### African countries

In [ ]:
response = requests.get("https://en.wikipedia.org/wiki/Category:Africa_country_and_territory_topics_templates")
page = response.text
soup = BeautifulSoup(page)

africa_templates = [(a.text, f'https://en.wikipedia.org{a["href"]}') for a in soup.find("div", id="mw-pages").find("div", class_="mw-content-ltr").find_all("a")]
africa_templates = africa_templates[1:4] + africa_templates[5:17] + africa_templates[18:42] + africa_templates[43:45] + africa_templates[46:51] + africa_templates[52:54] + africa_templates[55:]
for i, place in enumerate(africa_templates):
    print(i, place[0])

### Middle Eastern countries

In [ ]:
response = requests.get("https://en.wikipedia.org/wiki/Category:Middle_East_country_and_territory_topics_templates")
page = response.text
soup = BeautifulSoup(page)

middle_east_templates = [(a.text, f'https://en.wikipedia.org{a["href"]}') for a in soup.find("div", id="mw-pages").find("div", class_="mw-content-ltr").find_all("a")]
middle_east_templates = middle_east_templates[:3] + middle_east_templates[4:10] + middle_east_templates[11:12] + middle_east_templates[14:18] + middle_east_templates[19:]
for i, place in enumerate(middle_east_templates):
    print(i, place[0])


### Asian countries

In [ ]:
response = requests.get("https://en.wikipedia.org/wiki/Category:Asia_country_and_territory_topics_templates")
page = response.text
soup = BeautifulSoup(page)

asia_templates = [(a.text, f'https://en.wikipedia.org{a["href"]}') for a in soup.find("div", class_="mw-category-generated").find_all("a")]
asia_templates = [('Template:Philippines topics', 'https://en.wikipedia.org/wiki/Template:Philippines_topics')] + asia_templates[5:11] + asia_templates[12:16] + asia_templates[17:36] + asia_templates[37:39] + asia_templates[40:47]
for i, place in enumerate(asia_templates):
    print(i, place[0])

### European countries

In [ ]:
response = requests.get("https://en.wikipedia.org/wiki/Category:Europe_country_and_territory_topics_templates")
page = response.text
soup = BeautifulSoup(page)

europe_templates = [(a.text, f'https://en.wikipedia.org{a["href"]}') for a in soup.find("div", class_="mw-category-generated").find_all("a")]
europe_templates = [('Template:Bosnia and Herzegovina topics', 'https://en.wikipedia.org/wiki/Bosnia_and_Herzegovina_cuisine'), ('Template:Canary Islands topics', 'https://en.wikipedia.org/wiki/Canarian_cuisine'), ("Template:Balearic Islands topics", "https://en.wikipedia.org/wiki/Balearic_cuisine"), ("Template:Bouvet Island topics", "https://en.wikipedia.org/wiki/Bouvet_Island"), ("Template:South Georgia and the South Sandwich Islands topics", "https://en.wikipedia.org/wiki/South_Georgia_and_the_South_Sandwich_Islands")] + europe_templates[6:23] + europe_templates[24:27] + europe_templates[28:50] + europe_templates[51:54] + europe_templates[55:62] + europe_templates[63:80]
for i, place in enumerate(europe_templates):
    print(i, place[0])

### North American countries

In [ ]:
response = requests.get("https://en.wikipedia.org/wiki/Category:North_America_country_and_territory_topics_templates")
page = response.text
soup = BeautifulSoup(page)

north_america_templates = [(a.text, f'https://en.wikipedia.org{a["href"]}') for a in soup.find("div", class_="mw-category-generated").find_all("a")]
north_america_templates = north_america_templates[4:8] + north_america_templates[9:10] + north_america_templates[11:]
for i, place in enumerate(north_america_templates):
    print(i, place[0])

### Oceanic countries

In [ ]:
response = requests.get("https://en.wikipedia.org/wiki/Category:Oceania_country_and_territory_topics_templates")
page = response.text
soup = BeautifulSoup(page)

oceania_templates = [(a.text, f'https://en.wikipedia.org{a["href"]}') for a in soup.find("div", class_="mw-category-generated").find_all("a")]
oceania_templates = [('Template:Rapa Nui topics', 'https://en.wikipedia.org/wiki/Pascuense_cuisine'), ('Template:Christmas Island topics', 'https://en.wikipedia.org/wiki/Christmas_Island_cuisine'), ("Template:Mariana Islands topics", "https://en.wikipedia.org/wiki/Cuisine_of_the_Mariana_Islands"), ("Template:Niue topics", "https://en.wikipedia.org/wiki/Niuean_cuisine")] + oceania_templates[1:]
for i, place in enumerate(oceania_templates):
    print(i, place[0])

### South American countries

In [ ]:
response = requests.get("https://en.wikipedia.org/wiki/Category:South_America_country_and_territory_topics_templates")
page = response.text
soup = BeautifulSoup(page)

south_america_templates = [(a.text, f'https://en.wikipedia.org{a["href"]}') for a in soup.find("div", class_="mw-category-generated").find_all("a")]
south_america_templates = south_america_templates[3:]
for i, place in enumerate(south_america_templates):
    print(i, place[0])

### Central American countries

In [ ]:
response = requests.get("https://en.wikipedia.org/wiki/Category:Central_America_country_and_territory_topics_templates")
page = response.text
soup = BeautifulSoup(page)

central_america_templates = [(a.text, f'https://en.wikipedia.org{a["href"]}') for a in soup.find("div", class_="mw-category-generated").find_all("a")]
central_america_templates = central_america_templates[3:4] + central_america_templates[5:]
for i, place in enumerate(central_america_templates):
    print(i, place[0])

### Caribbean countries

In [ ]:
response = requests.get("https://en.wikipedia.org/wiki/Category:Caribbean_country_and_territory_topics_templates")
page = response.text
soup = BeautifulSoup(page)

caribbean_templates = [(a.text, f'https://en.wikipedia.org{a["href"]}') for a in soup.find("div", class_="mw-category-generated").find_all("a")]
caribbean_templates = caribbean_templates[3:]
for i, place in enumerate(caribbean_templates):
    print(i, place[0])

## Manually adding in Wiki page content for countries not accounted for above

In [ ]:
zimbabwe_raw = BeautifulSoup('''<div class="thumbinner" style="width:222px;"><a href="/wiki/File:Goat_Offal.JPG" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/d/d8/Goat_Offal.JPG/220px-Goat_Offal.JPG" decoding="async" width="220" height="165" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/d/d8/Goat_Offal.JPG/330px-Goat_Offal.JPG 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/d/d8/Goat_Offal.JPG/440px-Goat_Offal.JPG 2x" data-file-width="4352" data-file-height="3264"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Goat_Offal.JPG" class="internal" title="Enlarge"></a></div>A meal of sadza <i>(right)</i>, greens, and goat <a href="/wiki/Offal" title="Offal">offal</a>. The goat's small intestines are wrapped around small pieces of large intestines before cooking.</div></div><p>Like in many African countries, the majority of Zimbabweans depend on a few staple foods. "Mealie meal", also known as <a href="/wiki/Cornmeal" title="Cornmeal">cornmeal</a>, is used to prepare <i><a href="/wiki/Sadza" class="mw-redirect" title="Sadza">sadza</a></i> or <i>isitshwala</i>, as well as <a href="/wiki/Porridge" title="Porridge">porridge</a> known as <i>bota</i> or <i>ilambazi</i>. Sadza is made by mixing the cornmeal with water to produce a thick paste/porridge. After the paste has been cooking for several minutes, more cornmeal is added to thicken the paste.
</p><p><a href="/wiki/Graduation" title="Graduation">Graduations</a>, <a href="/wiki/Wedding" title="Wedding">weddings</a>, and any other family gatherings will usually be celebrated with the killing of a goat or cow, which will be barbecued or <a href="/wiki/Roasting" title="Roasting">roasted</a> by the family.
</p><p>This is usually eaten as lunch or dinner, usually with sides such as <a href="/wiki/Gravy" title="Gravy">gravy</a>, <a href="/wiki/Vegetable" title="Vegetable">vegetables</a> (<a href="/wiki/Spinach" title="Spinach">spinach</a>, <a href="/wiki/Chomolia" title="Chomolia">chomolia</a>, or <a href="/wiki/Spring_greens" title="Spring greens">spring greens</a>/<a href="/wiki/Collard_greens" class="mw-redirect" title="Collard greens">collard greens</a>), <a href="/wiki/Bean" title="Bean">beans</a>, and <a href="/wiki/Meat" title="Meat">meat</a> (stewed, grilled, roasted, or sundried). Sadza is also commonly eaten with curdled milk (<a href="/wiki/Sour_milk" class="mw-redirect" title="Sour milk">sour milk</a>), commonly known as "lacto" (<i>mukaka wakakora</i>), or dried <a href="/wiki/Tanganyika_sardine" class="mw-redirect" title="Tanganyika sardine">Tanganyika sardine</a>, known locally as <i>kapenta</i> or <i>matemba</i>. Bota is a thinner porridge, cooked without the additional cornmeal and usually flavoured with <a href="/wiki/Peanut_butter" title="Peanut butter">peanut butter</a>, milk, butter, or <a href="/wiki/Jam" class="mw-redirect" title="Jam">jam</a>.<sup id="cite_ref-263" class="reference"><a href="#cite_note-263">[263]</a></sup> Bota is usually eaten for breakfast.
</p><div class="thumb tright"><div class="thumbinner" style="width:222px;"><a href="/wiki/File:Boerewors_raw.jpg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/c/c6/Boerewors_raw.jpg/220px-Boerewors_raw.jpg" decoding="async" width="220" height="165" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/c/c6/Boerewors_raw.jpg/330px-Boerewors_raw.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/c/c6/Boerewors_raw.jpg/440px-Boerewors_raw.jpg 2x" data-file-width="450" data-file-height="338"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Boerewors_raw.jpg" class="internal" title="Enlarge"></a></div>Raw <a href="/wiki/Boerewors" title="Boerewors">boerewors</a></div></div></div><p>Even though the <a href="/wiki/Afrikaner" class="mw-redirect" title="Afrikaner">Afrikaners</a> are a small group (10%) within the white minority group, Afrikaner recipes are popular. <a href="/wiki/Biltong" title="Biltong">Biltong</a>, a type of <a href="/wiki/Jerky_(food)" class="mw-redirect" title="Jerky (food)">jerky</a>, is a popular snack, prepared by hanging bits of spiced raw meat to dry in the shade.<sup id="cite_ref-264" class="reference"><a href="#cite_note-264">[264]</a></sup> <a href="/wiki/Boerewors" title="Boerewors">Boerewors</a> is served with sadza. It is a long sausage, often well-spiced, composed of beef rather than pork, and barbecued.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (May 2016)">citation needed</span></a></i>]</sup> As Zimbabwe was a British colony, some people there have adopted some colonial-era English eating habits. For example, most people will have <a href="/wiki/Porridge" title="Porridge">porridge</a> in the morning, as well as 10 o'clock tea (midday tea). They will have lunch, often leftovers from the night before, freshly cooked <i>sadza</i>, or sandwiches (which is more common in the cities). After lunch, there is usually 4 o'clock tea (afternoon tea), which is served before dinner. It is not uncommon for <a href="/wiki/Tea" title="Tea">tea</a> to be had after dinner.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (May 2016)">citation needed</span></a></i>]</sup>
</p><p>Rice, pasta, and <a href="/wiki/Potato_production_in_Zimbabwe" title="Potato production in Zimbabwe">potato-based foods</a> (<a href="/wiki/French_fries" title="French fries">french fries</a> and <a href="/wiki/Mashed_potato" title="Mashed potato">mashed potato</a>) also make up part of Zimbabwean cuisine. A local favourite is rice cooked with peanut butter, which is taken with thick gravy, mixed vegetables and meat.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (May 2016)">citation needed</span></a></i>]</sup> A potpourri of peanuts known as <i>nzungu</i>, boiled and sundried maize, <a href="/wiki/Black-eyed_pea" title="Black-eyed pea">black-eyed peas</a> known as <i>nyemba</i>, and <a href="/wiki/Bambara_groundnut" class="mw-redirect" title="Bambara groundnut">bambara groundnuts</a> known as <i>nyimo</i> makes a traditional dish called <i>mutakura</i>. Mutakura can also be the above ingredients cooked individually. One can also find local snacks, such as <i>maputi</i> (roasted/popped maize kernels similar to <a href="/wiki/Popcorn" title="Popcorn">popcorn</a>), roasted and salted <a href="/wiki/Peanut" title="Peanut">peanuts</a>, <a href="/wiki/Sugar_cane" class="mw-redirect" title="Sugar cane">sugar cane</a>, <a href="/wiki/Sweet_potato" title="Sweet potato">sweet potato</a>, pumpkin, and indigenous fruits, such as <a href="/wiki/Horned_melon" class="mw-redirect" title="Horned melon">horned melon</a>, <i>gaka</i>, <i><a href="/wiki/Adansonia" title="Adansonia">adansonia</a></i>, <i>mawuyu</i>, <i><a href="/wiki/Uapaca_kirkiana" title="Uapaca kirkiana">uapaca kirkiana</a></i>, <i>mazhanje</i> (sugar plum), and many others.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (May 2016)">citation needed</span></a></i>]</sup>
</p>''')

soup = zimbabwe_raw

tags_to_extract = ['table', 'script', 'meta', 'style']
titles_to_extract = ['Jump up', 'Enlarge', re.compile("(Wikipedia:)"), "Help:Category", re.compile("(Category:Wiki.*)"), re.compile("(Category:Comm.*)"), re.compile("(Category:Articles.*)"), re.compile("(Category:All )"), 'ISBN_(identifier)', re.compile("(Special:)"), re.compile("(Category:CS1)")]
texts_to_extract = ['edit', re.compile("(Jump to)")]
classes_to_extract = ['reference']
ids_to_extract = ['mw-hidden-catlinks']

for t in tags_to_extract:
    [s.extract() for s in soup(t)]
for t in ids_to_extract:
    [s.extract() for s in soup(id=t)]
for t in titles_to_extract:
    [s.extract() for s in soup("a", title=t)]
for t in texts_to_extract:
    [s.extract() for s in soup("a", text=t)]
for t in classes_to_extract:
    [s.extract() for s in soup("a", class_=t)]

zimbabwe_links = [("Zimbabwe#Cuisine", "Zimbabwean cuisine")] + list(set([(wikilink["href"].lstrip('/wiki/'), wikilink.text) for wikilink in soup.find_all("a", href=re.compile("(^\/wiki\/.+)")) if wikilink.text != "" and wikilink.text != " "]))
zimbabwe_links

In [ ]:
kenyan_cuisine_raw = BeautifulSoup('''<div class="thumb tright"><div class="thumbinner" style="width:222px;"><a href="/wiki/File:Ugali_with_beef_and_sauce.JPG" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/6/65/Ugali_with_beef_and_sauce.JPG/220px-Ugali_with_beef_and_sauce.JPG" decoding="async" width="220" height="165" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/6/65/Ugali_with_beef_and_sauce.JPG/330px-Ugali_with_beef_and_sauce.JPG 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/6/65/Ugali_with_beef_and_sauce.JPG/440px-Ugali_with_beef_and_sauce.JPG 2x" data-file-width="2592" data-file-height="1944"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Ugali_with_beef_and_sauce.JPG" class="internal" title="Enlarge"></a></div><a href="/wiki/Ugali" title="Ugali">Ugali</a> is a mainstay of the cuisine throughout the <a href="/wiki/African_Great_Lakes" title="African Great Lakes">African Great Lakes</a> region. Served here with beef and sauce.</div></div></div><p>There is no singular dish that represents all of Kenya's wide cuisine. Different communities have their own native foods. Staples are maize and other cereals depending on the region, including <a href="/wiki/Millet" title="Millet">millet</a> and <a href="/wiki/Sorghum" title="Sorghum">sorghum</a> eaten with various meats and vegetables. The foods that are universally eaten in Kenya are <a href="/wiki/Ugali" title="Ugali">ugali</a>, <a href="/wiki/Sukuma_wiki" title="Sukuma wiki">sukuma wiki</a>, and nyama choma. Kenya's coastal cuisine is unique and highly regarded throughout the country.
</p><p>Sukuma wiki, a <a href="/wiki/Swahili_language" title="Swahili language">Swahili</a> phrase which literally means "to push the week", is a simple dish made with greens similar to <a href="/wiki/Kale" title="Kale">kale</a> or <a href="/wiki/Collard_greens" class="mw-redirect" title="Collard greens">collards</a> that can also be made with cassava leaves, sweet potato leaves, or pumpkin leaves. Its Swahili name comes from the fact that it is typically eaten to "get through the week" or "stretch the week". Nyama choma is grilled meat, which can be considered as an unofficial national dish  – usually goat or sheep - while kuku choma is grilled chicken. It is usually cooked over an open fire. It is usually eaten with <a href="/wiki/Ugali" title="Ugali">ugali</a> and <a href="/wiki/Kachumbari" title="Kachumbari">kachumbari</a>, <a href="/w/index.php?title=Maharagwe&amp;action=edit&amp;redlink=1" class="new" title="Maharagwe (page does not exist)">maharagwe</a> (bean stew) and <a href="/wiki/Mchicha" class="mw-redirect" title="Mchicha">mchicha</a> (shredded spinach).	
</p><p>Among the <a href="/wiki/Luhya_people" title="Luhya people">Luhya</a> residing in the western region of the country, <i>ingokho</i> (<a href="/wiki/Chicken_(food)" class="mw-redirect" title="Chicken (food)">chicken</a>) and ugali is a favorite meal. Other than these, they also eat tsisaka, miroo, managu and other dishes. In the <a href="/wiki/Rift_Valley_Province" title="Rift Valley Province">Rift Valley</a>, the <a href="/wiki/Kalenjin_people" title="Kalenjin people">Kalenjin</a> have long made <a href="/wiki/Mursik" title="Mursik">mursik</a>, which they have with <i>kimyet</i> (<a href="/wiki/Ugali" title="Ugali">ugali</a>) and a vegetable relish called <i>isageek</i>. Also among the <a href="/wiki/Kikuyu_people" title="Kikuyu people">Kikuyu</a> of Central Kenya, many <a href="/wiki/Tubers" class="mw-redirect" title="Tubers">tubers</a>, including <i>ngwaci</i> (<a href="/wiki/Sweet_potatoes" class="mw-redirect" title="Sweet potatoes">sweet potatoes</a>), <i>ndũma</i> (<a href="/wiki/Taro_root" class="mw-redirect" title="Taro root">taro root</a>, known in Kenya as <a href="/wiki/Arrowroot" title="Arrowroot">arrowroot</a>), <i>ikwa</i> (<a href="/wiki/Yam_(vegetable)" title="Yam (vegetable)">yams</a>), and <i>mianga</i> (<a href="/wiki/Cassava" title="Cassava">cassava</a>) are eaten, as well as legumes like <a href="/wiki/Beans" class="mw-redirect" title="Beans">beans</a> and a Kikuyu bean known as <i>njahi</i>. Among the <a href="/wiki/Luo_people_of_Kenya_and_Tanzania" class="mw-redirect" title="Luo people of Kenya and Tanzania">Luo</a> residing on the western region around Lake Victoria, "kuon" (Ugali) and "rech" fish is a favourite, as well as "gweno" (chicken), "Aliya" (sun dried meat), "Onyoso" (type of ant), "Ng'wen" (termitoidae), "Dede" (grasshoppers), various birds and green vegetables "Alode" such as "Osuga", "Akeyo", "Muto", "Dodo", "Dek", "Apoth" and "Bo", all consumed with Ugali.
</p><p>As you travel around the country distinct differences are noted mainly based on what foods are locally available around such areas. Grains are a staple food for groups that grow grains (e.g. Kikuyu, Embu, Meru, Kisii, etc.). Other communities such as the Luo and the Coastal community have <a href="/wiki/Fish_(food)" class="mw-redirect" title="Fish (food)">fish</a> and seafood for their staple food as available in such areas. In semi-arid areas like Turkana, foods made from sorghum are more common staple foods. As you move towards the city – food eaten by working families vary according to preference and ethnicity. Rice and stew is more common with working families, and other dishes like <a href="/wiki/Chapati" title="Chapati">chapati</a>  which is a staple that originated from India and is similar to the flatbread (parantha), chicken stew, etc.
</p>''')

soup = kenyan_cuisine_raw

tags_to_extract = ['table', 'script', 'meta', 'style']
titles_to_extract = ['Jump up', 'Enlarge', re.compile("(Wikipedia:)"), "Help:Category", re.compile("(Category:Wiki.*)"), re.compile("(Category:Comm.*)"), re.compile("(Category:Articles.*)"), re.compile("(Category:All )"), 'ISBN_(identifier)', re.compile("(Special:)"), re.compile("(Category:CS1)")]
texts_to_extract = ['edit', re.compile("(Jump to)")]
classes_to_extract = ['reference']
ids_to_extract = ['mw-hidden-catlinks']

for t in tags_to_extract:
    [s.extract() for s in soup(t)]
for t in ids_to_extract:
    [s.extract() for s in soup(id=t)]
for t in titles_to_extract:
    [s.extract() for s in soup("a", title=t)]
for t in texts_to_extract:
    [s.extract() for s in soup("a", text=t)]
for t in classes_to_extract:
    [s.extract() for s in soup("a", class_=t)]

kenya_links = [("Kenyan#Cuisine", "Kenyan cuisine")] + list(set([(wikilink["href"].lstrip('/wiki/'), wikilink.text) for wikilink in soup.find_all("a", href=re.compile("(^\/wiki\/.+)")) if wikilink.text != "" and wikilink.text != " "]))
kenya_links

In [ ]:
jersey_cuisine_raw = BeautifulSoup('''<div class="thumb tright"><div class="thumbinner" style="width:222px;"><a href="/wiki/File:Jersey_wonders.jpg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/1/14/Jersey_wonders.jpg/220px-Jersey_wonders.jpg" decoding="async" width="220" height="172" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/1/14/Jersey_wonders.jpg/330px-Jersey_wonders.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/1/14/Jersey_wonders.jpg/440px-Jersey_wonders.jpg 2x" data-file-width="1402" data-file-height="1098"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Jersey_wonders.jpg" class="internal" title="Enlarge"></a></div><i>Jersey wonders</i>, or <i>mèrvelles</i>, are a favourite snack consisting of fried dough, found especially at country fêtes. According to tradition, the success of cooking depends on the state of the tide.</div></div></div><p>Seafood has traditionally been important to the cuisine of Jersey: <a href="/wiki/Mussel" title="Mussel">mussels</a> (called <i>moules</i> in the island), <a href="/wiki/Oyster" title="Oyster">oysters</a>, <a href="/wiki/European_lobster" class="mw-redirect" title="European lobster">lobster</a> and crabs – especially <a href="/wiki/Maja_squinado" title="Maja squinado">spider crabs</a> – <a href="/wiki/Abalone" title="Abalone">ormers</a> and <a href="/wiki/Conger" title="Conger">conger</a>.
</p><p>Jersey milk being very rich, <a href="/wiki/Cream" title="Cream">cream</a> and butter have played a large part in insular cooking. <i>(See <a href="/wiki/Channel_Island_milk" title="Channel Island milk">Channel Island milk</a>)</i> However, there is no indigenous tradition of cheese making, contrary to the custom of mainland Normandy, but some cheese is produced commercially. Jersey <a href="/wiki/Fudge" title="Fudge">fudge</a>, mostly imported and made with milk from overseas Jersey cattle herds, is a popular food product with tourists.
</p><p><a href="/wiki/Jersey_Royal_potatoes" class="mw-redirect" title="Jersey Royal potatoes">Jersey Royal potatoes</a> are the local variety of <a href="/wiki/New_potato" class="mw-redirect" title="New potato">new potato</a>, and the island is famous for its early crop of Chats (small potatoes) from the south-facing côtils (steeply sloping fields). They were originally grown using <a href="/wiki/Seaweed_fertiliser" title="Seaweed fertiliser">vraic</a> as a natural fertiliser giving them their own individual taste, only a small portion of those grown in the island still use this method. They are eaten in a variety of ways, often simply boiled and served with butter or when not as fresh fried in butter.
</p><p>Apples historically were an important crop. <i>Bourdélots</i> are apple dumplings, but the most typical speciality is black butter (<i><a href="/wiki/Ni%C3%A8r_beurre" title="Nièr beurre">lé nièr beurre</a></i>), a dark spicy spread prepared from apples, cider and spices. <a href="/wiki/Cider" title="Cider">Cider</a> used to be an important export. After decline and near-disappearance in the late 20th century, apple production is being increased and promoted. Besides cider, <a href="/wiki/Apple_brandy" class="mw-redirect" title="Apple brandy">apple brandy</a> is produced. Other production of alcohol drinks includes wine,<sup id="cite_ref-105" class="reference"><a href="#cite_note-105">[105]</a></sup>  and in 2013 the first commercial <a href="/wiki/Vodka" title="Vodka">vodkas</a> made from Jersey Royal potatoes were marketed.<sup id="cite_ref-106" class="reference"><a href="#cite_note-106">[106]</a></sup>
</p><p>Among other traditional dishes are cabbage loaf, Jersey wonders (<i>les mèrvelles</i>), fliottes, bean crock (<i>les pais au fou</i>), <a href="/wiki/Urtica_dioica" title="Urtica dioica">nettle</a> (<i>ortchie</i>) soup, vraic buns.
</p>''')

soup = jersey_cuisine_raw

tags_to_extract = ['table', 'script', 'meta', 'style']
titles_to_extract = ['Jump up', 'Enlarge', re.compile("(Wikipedia:)"), "Help:Category", re.compile("(Category:Wiki.*)"), re.compile("(Category:Comm.*)"), re.compile("(Category:Articles.*)"), re.compile("(Category:All )"), 'ISBN_(identifier)', re.compile("(Special:)"), re.compile("(Category:CS1)")]
texts_to_extract = ['edit', re.compile("(Jump to)")]
classes_to_extract = ['reference']
ids_to_extract = ['mw-hidden-catlinks']

for t in tags_to_extract:
    [s.extract() for s in soup(t)]
for t in ids_to_extract:
    [s.extract() for s in soup(id=t)]
for t in titles_to_extract:
    [s.extract() for s in soup("a", title=t)]
for t in texts_to_extract:
    [s.extract() for s in soup("a", text=t)]
for t in classes_to_extract:
    [s.extract() for s in soup("a", class_=t)]

jersey_links = [("Jersey#Food_and_drink", "Jersey cuisine")] + list(set([(wikilink["href"].lstrip('/wiki/'), wikilink.text) for wikilink in soup.find_all("a", href=re.compile("(^\/wiki\/.+)")) if wikilink.text != "" and wikilink.text != " "]))
jersey_links

In [ ]:
cook_islands_cuisine_raw = BeautifulSoup('''<p>Due to the island location and the fact that the Cook Islands produce a significant array of fruits and vegetables, natural local produce, especially coconut, features in many of the dishes of the islands as does fresh seafood. While most food is imported from New Zealand, there are several Growers' Associations, such as Mangaian, Ngatangiia, Penrhyn, Puaikura, and Rakahanga, which contribute produce for local cuisine.<sup id="cite_ref-FOOTNOTECrocombe199077_26-0" class="reference"><a href="#cite_note-FOOTNOTECrocombe199077-26">[26]</a></sup> Typical local cuisine includes <a href="/wiki/Arrowroot" title="Arrowroot">arrowroot</a>, <a href="/wiki/Clam" title="Clam">clams</a>, octopus, and <a href="/wiki/Taro" title="Taro">taro</a>, and seasonings such as fresh <a href="/wiki/Ginger" title="Ginger">ginger</a>, <a href="/wiki/Lime_(fruit)" title="Lime (fruit)">lime</a>, lemon, <a href="/wiki/Basil" title="Basil">basil</a>, garlic and <a href="/wiki/Coconut" title="Coconut">coconut</a>.<sup id="cite_ref-FOOTNOTEPignéguyRaela2011_27-0" class="reference"><a href="#cite_note-FOOTNOTEPignéguyRaela2011-27">[27]</a></sup> <i>Rukau</i> is a dish of taro leaves cooked with coconut sauce and onion.  A meal of octopus is known locally as <i>Eke</i>, and <a href="/wiki/Suckling_pig" title="Suckling pig">suckling pig</a> is known as <i>Puaka</i>.<sup id="cite_ref-FOOTNOTEHunt200346_28-0" class="reference"><a href="#cite_note-FOOTNOTEHunt200346-28">[28]</a></sup> <i>Ika mata</i> is a dish of raw fish marinated with lemon or lime and served with coconut cream,<sup id="cite_ref-FOOTNOTEHunt200346_28-1" class="reference"><a href="#cite_note-FOOTNOTEHunt200346-28">[28]</a></sup> while <i>Pai Ika</i> and <i>Keke Ika</i> are also local fish fare.<sup id="cite_ref-FOOTNOTEMatenga-Smith199077_29-0" class="reference"><a href="#cite_note-FOOTNOTEMatenga-Smith199077-29">[29]</a></sup> <i>Poke</i> is a dessert which can be made in one of two ways, either with banana and coconut milk or with <a href="/wiki/Papaya" title="Papaya">pawpaw</a>.<sup id="cite_ref-FOOTNOTEFoldPritchard200586_30-0" class="reference"><a href="#cite_note-FOOTNOTEFoldPritchard200586-30">[30]</a></sup> <a href="/wiki/Soursop" title="Soursop">Soursop</a>, oranges and mangos are popular as juices.<sup id="cite_ref-FOOTNOTEMatenga-Smith199082_31-0" class="reference"><a href="#cite_note-FOOTNOTEMatenga-Smith199082-31">[31]</a></sup> <a href="/wiki/Coconut_water" title="Coconut water">Coconut water</a>, local beer (Cooks Lager), and coffee are popular beverages among the Cook Islanders.<sup id="cite_ref-FOOTNOTEHunt200346_28-2" class="reference"><a href="#cite_note-FOOTNOTEHunt200346-28">[28]</a></sup>
</p>''')

soup = cook_islands_cuisine_raw

tags_to_extract = ['table', 'script', 'meta', 'style']
titles_to_extract = ['Jump up', 'Enlarge', re.compile("(Wikipedia:)"), "Help:Category", re.compile("(Category:Wiki.*)"), re.compile("(Category:Comm.*)"), re.compile("(Category:Articles.*)"), re.compile("(Category:All )"), 'ISBN_(identifier)', re.compile("(Special:)"), re.compile("(Category:CS1)")]
texts_to_extract = ['edit', re.compile("(Jump to)")]
classes_to_extract = ['reference']
ids_to_extract = ['mw-hidden-catlinks']

for t in tags_to_extract:
    [s.extract() for s in soup(t)]
for t in ids_to_extract:
    [s.extract() for s in soup(id=t)]
for t in titles_to_extract:
    [s.extract() for s in soup("a", title=t)]
for t in texts_to_extract:
    [s.extract() for s in soup("a", text=t)]
for t in classes_to_extract:
    [s.extract() for s in soup("a", class_=t)]

cook_islands_links = [("Culture_of_the_Cook_Islands#Cuisine", "Cook Islands cuisine")] + list(set([(wikilink["href"].lstrip('/wiki/'), wikilink.text) for wikilink in soup.find_all("a", href=re.compile("(^\/wiki\/.+)")) if wikilink.text != "" and wikilink.text != " "]))
cook_islands_links

In [ ]:
tongan_cuisine_raw = BeautifulSoup('''<div class="thumb tright"><div class="thumbinner" style="width:202px;"><a href="/wiki/File:Tupenu02.JPG" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/en/thumb/9/95/Tupenu02.JPG/200px-Tupenu02.JPG" decoding="async" width="200" height="189" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/en/thumb/9/95/Tupenu02.JPG/300px-Tupenu02.JPG 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/9/95/Tupenu02.JPG/400px-Tupenu02.JPG 2x" data-file-width="1008" data-file-height="952"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Tupenu02.JPG" class="internal" title="Enlarge"></a></div>Women cooking <i>topai</i> for mourners.</div></div></div><p>In former times, there was only one main meal, a midday meal cooked in an earth oven. Villagers would rise, eat some leftover food from the previous day's meal, and set out to work in the fields, <a href="/wiki/Fishing" title="Fishing">fishing</a>, gathering <a href="/wiki/Shellfish" title="Shellfish">shellfish</a>, etc. The results of the morning's work would be cooked by the men, and served to the assembled household. The remnants would be placed in a basket suspended from a tree. This food is served as an end-of-the-day snack as well as the next day's breakfast. Food past its prime was given to the pigs.
</p><p>The diet consisted mainly of <a href="/wiki/Taro" title="Taro">taro</a>, <a href="/wiki/Oxalis_tuberosa" title="Oxalis tuberosa">yams</a>, <a href="/wiki/Bananas" class="mw-redirect" title="Bananas">bananas</a>, <a href="/wiki/Coconuts" class="mw-redirect" title="Coconuts">coconuts</a>, and <a href="/wiki/Fish" title="Fish">fish</a> baked in leaves; shellfish were usually served raw, as a <a href="/wiki/Relish" title="Relish">relish</a>. The liquid from the center of coconuts was commonly drunk, and the soft "spoon meat" of young coconuts much relished. Baked <a href="/wiki/Breadfruit" title="Breadfruit">breadfruit</a> was eaten in season. <a href="/wiki/Pigs" class="mw-redirect" title="Pigs">Pigs</a> were killed and cooked only on special occasions, such as <a href="/wiki/Weddings" class="mw-redirect" title="Weddings">weddings</a>, <a href="/wiki/Funerals" class="mw-redirect" title="Funerals">funerals</a>, <a href="/wiki/Feasts" class="mw-redirect" title="Feasts">feasts</a> honoring a visiting chief, and the like. Tongans also ate <a href="/wiki/Chickens" class="mw-redirect" title="Chickens">chickens</a>.
</p><p>Food could be stored by feeding it to pigs. Pre-contact Tongans also built elevated storehouses for yams. Yams would keep only a few months. Hence a household's main security was generous distribution of food to relatives and neighbors, who were thus put under an obligation to share in their turn.
</p><p>Many new foods were introduced in the 19th and early 20th centuries, following Western contacts and settlements. The <a href="/wiki/Cassava" title="Cassava">cassava</a> plant was one such introduction; it is called <i>manioke</i> in Tongan. While it lacks the prestige of the yam, it is an easy plant to grow and a common crop. Introduced <a href="/wiki/Watermelons" class="mw-redirect" title="Watermelons">watermelons</a> became popular. They were eaten either by themselves, or pulped and mixed with <a href="/wiki/Coconut_milk" title="Coconut milk">coconut milk</a>, forming a popular drink called <i><a href="/wiki/%27otai" title="'otai">'otai</a></i>. Other fruits, such as <a href="/wiki/Orange_(fruit)" title="Orange (fruit)">oranges</a>, <a href="/wiki/Lemon" title="Lemon">lemons</a>, and <a href="/wiki/Lime_(fruit)" title="Lime (fruit)">limes</a>, became popular. Tongans also adopted <a href="/wiki/Onion" title="Onion">onions</a>, <a href="/wiki/Scallion" title="Scallion">green onions</a>, <a href="/wiki/Cabbage" title="Cabbage">cabbage</a>, <a href="/wiki/Carrot" title="Carrot">carrots</a>, <a href="/wiki/Tomato" title="Tomato">tomatoes</a>, and other common vegetables. In the last few decades, Tongan farmers with access to large tracts of land have engaged in commercial farming of <a href="/wiki/Pumpkin" title="Pumpkin">pumpkins</a> and other easily shipped vegetables as cash crops.
</p><p>Tongans now consume large quantities of imported <a href="/wiki/Flour" title="Flour">flour</a> and <a href="/wiki/Sugar" title="Sugar">sugar</a>. One dish that uses both is <i>topai</i> (doughboys), flour and water worked into a paste and dropped into a kettle of boiling water, then served with a <a href="/wiki/Syrup" title="Syrup">syrup</a> of sugar and coconut milk. Topai are a common funeral food, being easily prepared for hundreds of mourners.
</p><p>There are now bakeries in the larger cities. The most popular loaves are soft, white, and bland. There are also local soft drink bottlers, who make various local varieties of <a href="/wiki/Soft_drink" title="Soft drink">soda</a>. A Tongan who might once have breakfasted on bits of cooked <a href="/wiki/Pork" title="Pork">pork</a> and yam from a hanging basket may now have <a href="/wiki/White_bread" title="White bread">white bread</a> and soda for breakfast.
</p><p>Purchased prepared foods have also made great headway, even in remote villages. Canned <a href="/wiki/Cornbeef" class="mw-redirect" title="Cornbeef">cornbeef</a> is a great favorite. It is eaten straight from the can, or mixed with coconut milk and onions, wrapped in leaves, and baked in the earth oven. Tongans also eat canned fish, such as <a href="/wiki/Tuna" title="Tuna">tuna</a>. In villages or towns with refrigeration, cheap frozen "<a href="/wiki/Mutton_flaps" title="Mutton flaps">mutton flaps</a>" imported from New Zealand are popular. Tongans also eat the common South Pacific "ship's biscuit", hard plain <a href="/wiki/Cracker_(food)" title="Cracker (food)">crackers</a> once a shipboard staple. These crackers are called <i>mā pakupaku</i>.
</p><p>Tongans no longer make an earth oven every day. Most daily cooking is done by women, who cook in battered pots over open fires in the village, in wood-burning stoves in some households, and on gas or electric ranges in some of the larger towns. The meal schedule has also changed, to more Westernized breakfast, light lunch, and heavy dinner. Tongans say that the old schedule is unworkable when household members have Western-style jobs, or attend schools at some distance from home; such family members cannot come home to eat, then have a doze after a heavy mid-day meal.
</p><p>As well as drinking soda, Tongans now drink <a href="/wiki/Tea" title="Tea">tea</a> and <a href="/wiki/Coffee" title="Coffee">coffee</a>. Usually this is of the cheapest variety, and served with tinned <a href="/wiki/Condensed_milk" title="Condensed milk">condensed milk</a>.
</p><p>Some men drink <a href="/wiki/Alcohol_(drug)" title="Alcohol (drug)">alcohol</a>. Sometimes this is imported Australian or New Zealand <a href="/wiki/Beer" title="Beer">beer</a>; more often it is home-brew, <i><a href="/wiki/Hopi" title="Hopi">hopi</a></i>, made with <a href="/wiki/Water" title="Water">water</a>, sugar or mashed <a href="/wiki/Fruit" title="Fruit">fruit</a>, and <a href="/wiki/Yeast" title="Yeast">yeast</a>. Imported drinks are sold only to Tongans who have liquor permits, which require a visit to a government office, and limit the amount of alcohol which can be purchased. There are no such formalities with <i>hopi</i>. Drinking is usually done secretively; a group of men gather and drink until they are drunk. Such gatherings sometimes result in drunken quarrels and assaults.
</p>''')

soup = tongan_cuisine_raw

tags_to_extract = ['table', 'script', 'meta', 'style']
titles_to_extract = ['Jump up', 'Enlarge', re.compile("(Wikipedia:)"), "Help:Category", re.compile("(Category:Wiki.*)"), re.compile("(Category:Comm.*)"), re.compile("(Category:Articles.*)"), re.compile("(Category:All )"), 'ISBN_(identifier)', re.compile("(Special:)"), re.compile("(Category:CS1)")]
texts_to_extract = ['edit', re.compile("(Jump to)")]
classes_to_extract = ['reference']
ids_to_extract = ['mw-hidden-catlinks']

for t in tags_to_extract:
    [s.extract() for s in soup(t)]
for t in ids_to_extract:
    [s.extract() for s in soup(id=t)]
for t in titles_to_extract:
    [s.extract() for s in soup("a", title=t)]
for t in texts_to_extract:
    [s.extract() for s in soup("a", text=t)]
for t in classes_to_extract:
    [s.extract() for s in soup("a", class_=t)]

tonga_links = [("Culture_of_Tonga#Cuisine", "Tongan cuisine")] + list(set([(wikilink["href"].lstrip('/wiki/'), wikilink.text) for wikilink in soup.find_all("a", href=re.compile("(^\/wiki\/.+)")) if wikilink.text != "" and wikilink.text != " "]))
tonga_links

In [ ]:
st_kitts_cuisine_raw = BeautifulSoup('''<p>With its rich soil, St. Kitts and Nevis grow a wide variety of fresh <a href="/wiki/Produce" title="Produce">produce</a>.  Abundant <a href="/wiki/Seafood" title="Seafood">seafood</a> and <a href="/wiki/Meat" title="Meat">meats</a> such as <a href="/wiki/Goat" title="Goat">goat</a> add to the diet.  The style of cooking is fairly simple, flavored much like other <a href="/wiki/West_Indian" title="West Indian">West Indian</a> cuisine.  Goat water <a href="/wiki/Stew" title="Stew">stew</a>, perhaps the country's most well-known dish, mixes goat, <a href="/wiki/Breadfruit" title="Breadfruit">breadfruit</a>, green pawpaw (<a href="/wiki/Papaya" title="Papaya">papaya</a>), and <a href="/wiki/Dumpling" title="Dumpling">dumplings</a> (also known as "droppers") in a tomato-based stew. Another favorite dish is cook-up, or <a href="/wiki/Pelau" title="Pelau">pelau</a>, which combines <a href="/wiki/Chicken" title="Chicken">chicken</a>, pig tail, <a href="/wiki/Saltfish" class="mw-redirect" title="Saltfish">saltfish</a> and <a href="/wiki/Vegetable" title="Vegetable">vegetables</a> with <a href="/wiki/Rice" title="Rice">rice</a> and <a href="/wiki/Pigeon_pea" title="Pigeon pea">pigeon peas</a>.  Conkies bear a large similarity to <a href="/wiki/Tamales" class="mw-redirect" title="Tamales">tamales</a>, though instead of having filling rolled inside the dough, the cornmeal is mixed together with grated sweet potato, pumpkin, coconut, and a few other ingredients; after wrapping the dough in banana leaves, they're boiled rather than steamed. Sweets tend to be simply made, sometimes with nothing more than fruit, like tamarind or guava, and sugar.
</p><p><a href="/wiki/Rum" title="Rum">Rum</a> is as popular on St. Kitts and Nevis as it is throughout the Caribbean.  The Brinley Gold Company manufactures rum on St. Kitts, with such distinctive flavors as coffee, mango, and vanilla.  But the national drink is actually Cane Spirits Rothschild (often abbreviated to CSR), distilled from fresh sugar cane. Belmont Estate and St. Kitts Rum also make rum on the island. In addition several of the beach bars will provide moonshine rum produced by individuals with homemade stills. 
</p><p>Many villages on Nevis hold cookouts on Friday and Saturday nights, where people come together to eat, drink, play games like <a href="/wiki/Dominoes" title="Dominoes">dominoes</a>, and have a good time.
</p>''')

soup = st_kitts_cuisine_raw

tags_to_extract = ['table', 'script', 'meta', 'style']
titles_to_extract = ['Jump up', 'Enlarge', re.compile("(Wikipedia:)"), "Help:Category", re.compile("(Category:Wiki.*)"), re.compile("(Category:Comm.*)"), re.compile("(Category:Articles.*)"), re.compile("(Category:All )"), 'ISBN_(identifier)', re.compile("(Special:)"), re.compile("(Category:CS1)")]
texts_to_extract = ['edit', re.compile("(Jump to)")]
classes_to_extract = ['reference']
ids_to_extract = ['mw-hidden-catlinks']

for t in tags_to_extract:
    [s.extract() for s in soup(t)]
for t in ids_to_extract:
    [s.extract() for s in soup(id=t)]
for t in titles_to_extract:
    [s.extract() for s in soup("a", title=t)]
for t in texts_to_extract:
    [s.extract() for s in soup("a", text=t)]
for t in classes_to_extract:
    [s.extract() for s in soup("a", class_=t)]

st_kitts_links = [("Culture_of_Saint_Kitts_and_Nevis#Cuisine", "Cuisine of Saint Kitts and Nevis")] + list(set([(wikilink["href"].lstrip('/wiki/'), wikilink.text) for wikilink in soup.find_all("a", href=re.compile("(^\/wiki\/.+)")) if wikilink.text != "" and wikilink.text != " "]))
st_kitts_links

In [ ]:
martinique_cuisine_raw = BeautifulSoup('''<p>As one would expect, <a href="/wiki/French_cuisine" title="French cuisine">French</a> and <a href="/wiki/Caribbean_cuisine" title="Caribbean cuisine">Creole cuisine</a> dominate Martinique's culinary landscape.  The two styles also combine by using French techniques with local produce, such as <a href="/wiki/Breadfruit" title="Breadfruit">breadfruit</a>, <a href="/wiki/Cassava" title="Cassava">cassava</a>, and <a href="/wiki/Christophine" class="mw-redirect" title="Christophine">christophine</a> (chayote).  Creole dishes rely heavily on <a href="/wiki/Seafood" title="Seafood">seafood</a>, including <a href="/wiki/Curry" title="Curry">curries</a> and <a href="/wiki/Fritter" title="Fritter">fritters</a>.  An exception is <a href="/wiki/Boudin" title="Boudin">Boudin</a>, a Creole type of blood sausage.  A dash of Chien sauce (made from onions, shallots, peppers, oil, and vinegar) adds a spicy touch to meals.  The favored island drink, 'Ti punch,' is a mixture of five parts of white rum to one part <a href="/wiki/Sugarcane" title="Sugarcane">sugarcane</a> syrup.  <a href="/wiki/Cr%C3%AAperie" class="mw-redirect" title="Crêperie">Crêperies</a>, <a href="/wiki/Brasserie" title="Brasserie">brasseries</a>, and restaurants featuring cuisine from various French regions can be found all over Martinique.
</p>''')

soup = martinique_cuisine_raw

tags_to_extract = ['table', 'script', 'meta', 'style']
titles_to_extract = ['Jump up', 'Enlarge', re.compile("(Wikipedia:)"), "Help:Category", re.compile("(Category:Wiki.*)"), re.compile("(Category:Comm.*)"), re.compile("(Category:Articles.*)"), re.compile("(Category:All )"), 'ISBN_(identifier)', re.compile("(Special:)"), re.compile("(Category:CS1)")]
texts_to_extract = ['edit', re.compile("(Jump to)")]
classes_to_extract = ['reference']
ids_to_extract = ['mw-hidden-catlinks']

for t in tags_to_extract:
    [s.extract() for s in soup(t)]
for t in ids_to_extract:
    [s.extract() for s in soup(id=t)]
for t in titles_to_extract:
    [s.extract() for s in soup("a", title=t)]
for t in texts_to_extract:
    [s.extract() for s in soup("a", text=t)]
for t in classes_to_extract:
    [s.extract() for s in soup("a", class_=t)]

martinique_links = [("Culture_of_Martinique#Cuisine", "Cuisine of Martinique")] + list(set([(wikilink["href"].lstrip('/wiki/'), wikilink.text) for wikilink in soup.find_all("a", href=re.compile("(^\/wiki\/.+)")) if wikilink.text != "" and wikilink.text != " "]))
martinique_links

In [ ]:
raw_html = '''<p><i><a href="/wiki/Kava" title="Kava">Kava</a></i> is the indigenous Polynesian beverage which is customarily served in all religious rituals. Kava is brewed in a wooden vessel known as a <i>tana</i>, a multilegged bowl which is also an art form carved in wood, that is much in demand.<sup id="cite_ref-Ibpus.comUSA2012_1-4" class="reference"><a href="#cite_note-Ibpus.comUSA2012-1">[1]</a></sup> Australian beer is also popular. Diet includes chicken, coconuts, <a href="/wiki/Common_fig" title="Common fig">figs</a>, <a href="/wiki/Mango" title="Mango">mangoes</a>, <a href="/wiki/Pandanus" title="Pandanus">pandanus</a>, pork, and wild cherry.
</p>'''

wallis_and_futuna_links = food.get_links_from_raw_html(raw_html, "Culture_of_Wallis_and_Futuna#Food_and_beverage", "Wallis and Futuna")
wallis_and_futuna_links

In [ ]:
country_cuisine_dict = food.get_cuisine_dict([africa_templates, middle_east_templates, asia_templates, europe_templates, north_america_templates, oceania_templates, south_america_templates, central_america_templates, caribbean_templates], kenya_links, zimbabwe_links, jersey_links, cook_islands_links, tonga_links, st_kitts_links, martinique_links, wallis_and_futuna_links)

In [ ]:
country_cuisine_df = pd.DataFrame.from_dict(country_cuisine_dict, orient='index', columns=["Continent", "Cuisine links", "Bottom nav links"])
country_cuisine_df.drop(columns="Bottom nav links", inplace=True)
country_cuisine_df

In [ ]:
with open("cuisines_df.pickle", "wb") as to_write:
    pickle.dump(country_cuisine_df, to_write)

# Building regional cuisine df
Compiling a df with data for Wikipedia pages (many but not all) that refer to a regional cuisine.

In [ ]:
regional_cuisine_df = pd.DataFrame(columns=["Places", "Links"])
regional_cuisine_df

In [ ]:
raw_html = '''<div class="thumb tright"><div class="thumbinner" style="width:302px;"><a href="/wiki/File:Madesu.jpg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/c/cb/Madesu.jpg/300px-Madesu.jpg" decoding="async" width="300" height="244" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/c/cb/Madesu.jpg/450px-Madesu.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/c/cb/Madesu.jpg/600px-Madesu.jpg 2x" data-file-width="1242" data-file-height="1012"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Madesu.jpg" class="internal" title="Enlarge"></a></div><i><a href="/w/index.php?title=Madesu&amp;action=edit&amp;redlink=1" class="new" title="Madesu (page does not exist)">Madesu</a></i> beans are a staple in Central Africa</div></div></div><p><br>
<a href="/wiki/Central_Africa" title="Central Africa">Central Africa</a> stretches from the <a href="/wiki/Tibesti_Mountains" title="Tibesti Mountains">Tibesti Mountains</a> in the north to the vast <a href="/wiki/Rainforest" title="Rainforest">rainforest</a> basin of the <a href="/wiki/Congo_River" title="Congo River">Congo River</a>, the highlands of Kivu and the savana of Katanga. This region has received culinary influence of the <a href="/wiki/Swahili_people" title="Swahili people">Swahilis</a> (a culture that evolved via the combination of Bantu, Yemeni, Omani and Indian cultures) during the East African Slave Trade. Swahili culinary influences can be found in dishes such as: mandanzi, pilaf rice, kachumbari, <a href="/wiki/Samosa" title="Samosa">sambsusa</a>, kuku paka, etc.<sup id="cite_ref-6" class="reference"><a href="#cite_note-6">[6]</a></sup> Central African cuisine has also been influenced by the Portuguese, by way of the <a href="/wiki/Kingdom_of_Kongo" title="Kingdom of Kongo">Kongo</a> and <a href="/wiki/Ndongo" class="mw-redirect" title="Ndongo">Ndongo</a> Kingdoms. Salt fish was introduced following trade in the late XVIIth and the Kikongo term for salt fish, makayabu, comes from the term bacalhau (ba-cal-ha-u).<sup id="cite_ref-7" class="reference"><a href="#cite_note-7">[7]</a></sup> The Portuguese culinary influence is especially prominent in Angola, Sao Tomé and Equatorial Guinea. Central Africa has also been influenced by the cuisine of the regions East, West and Southern Africa because of their close proximity. I.e: Babuté/<a href="/wiki/Bobotie" title="Bobotie">Bobotie</a> is shared with the south, nyama choma with the east and gombos with West Africa.
</p><div class="thumb tright"><div class="thumbinner" style="width:302px;"><a href="/wiki/File:Ntaba.jpg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/f/fb/Ntaba.jpg/300px-Ntaba.jpg" decoding="async" width="300" height="302" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/f/fb/Ntaba.jpg/450px-Ntaba.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/f/fb/Ntaba.jpg 2x" data-file-width="599" data-file-height="602"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Ntaba.jpg" class="internal" title="Enlarge"></a></div><i>Ntaba</i> Central African-style lamb barbecue</div></div></div><p>The main ingredients are <a href="/wiki/Plantain_(cooking)" class="mw-redirect" title="Plantain (cooking)">plantains</a>, <a href="/wiki/Cassava" title="Cassava">cassava</a>, rice, kwanga (cassava dumpling) and <a href="/wiki/Yam_(vegetable)" title="Yam (vegetable)">yam</a>. <a href="/wiki/Fufu" title="Fufu">Fufu</a>-like starchy foods are usually made from fermented cassava roots, but they can also be made with plantain, corn maize and yam. Fufu is served buffet style with grilled meat, fish, stews, greens and <a href="/wiki/Pili_pili" class="mw-redirect" title="Pili pili">piment</a> . A variety of local ingredients are used while preparing other dishes like <a href="/wiki/Spinach" title="Spinach">spinach</a> stew cooked with tomato, peppers, chillis, onions, and peanut butter.<sup id="cite_ref-8" class="reference"><a href="#cite_note-8">[8]</a></sup> Eastern central Africa is also one of the few regions in Africa that uses potatoes as one of its main bases, since potatoes grow easily in the region.
</p><p>Cassava plants are also consumed as cooked <a href="/wiki/Leaf_vegetable" title="Leaf vegetable">greens</a>. Groundnut (peanut) stew is also prepared, containing <a href="/wiki/Chicken_(food)" class="mw-redirect" title="Chicken (food)">chicken</a>, <a href="/wiki/Okra" title="Okra">okra</a>, <a href="/wiki/Ginger" title="Ginger">ginger</a>, and other spices. Another commonly served dish is bambara, a <a href="/wiki/Porridge" title="Porridge">porridge</a> of rice, peanut butter and sugar. Chicken and fish are favorite meat dishes, but <a href="/wiki/Bushmeat" title="Bushmeat">game meat</a> preparations containing <a href="/wiki/Crocodile" title="Crocodile">crocodile</a>, <a href="/wiki/Elephant" title="Elephant">elephant</a>, <a href="/wiki/Antelope" title="Antelope">antelope</a> and <a href="/wiki/Warthog" title="Warthog">warthog</a> are also served occasionally.<sup id="cite_ref-Huchzermeyer_2003_p._130_9-0" class="reference"><a href="#cite_note-Huchzermeyer_2003_p._130-9">[9]</a></sup><sup id="cite_ref-Daniel_Stiles_p._36_10-0" class="reference"><a href="#cite_note-Daniel_Stiles_p._36-10">[10]</a></sup><sup id="cite_ref-Stiles_2011_p._25_11-0" class="reference"><a href="#cite_note-Stiles_2011_p._25-11">[11]</a></sup><sup id="cite_ref-Whitford_1877_p._212_12-0" class="reference"><a href="#cite_note-Whitford_1877_p._212-12">[12]</a></sup><sup id="cite_ref-Gibbons_1898_p._223_13-0" class="reference"><a href="#cite_note-Gibbons_1898_p._223-13">[13]</a></sup>
</p>'''


central_africa = ["Angola", "Cameroon", "Central African Republic", "Chad", "Democratic Republic of the Congo", "Republic of the Congo", "Equatorial Guinea", "Gabon", "São Tomé and Príncipe"]
central_african_links = food.get_links_from_raw_html(raw_html, "African_cuisine#Central_Africa", "Central Africa")
regional_cuisine_df = pd.DataFrame.from_dict({central_african_links[0][1]: [central_africa, central_african_links]}, orient='index', columns=["Places", "Links"])
regional_cuisine_df

In [ ]:
raw_html = '''<div class="thumb tright"><div class="thumbinner" style="width:202px;"><a href="/wiki/File:Ugali_and_cabbage.jpg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/f/fe/Ugali_and_cabbage.jpg/200px-Ugali_and_cabbage.jpg" decoding="async" width="200" height="175" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/f/fe/Ugali_and_cabbage.jpg/300px-Ugali_and_cabbage.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/f/fe/Ugali_and_cabbage.jpg/400px-Ugali_and_cabbage.jpg 2x" data-file-width="2183" data-file-height="1912"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Ugali_and_cabbage.jpg" class="internal" title="Enlarge"></a></div><i><a href="/wiki/Ugali" title="Ugali">Ugali</a></i> is pictured here with a side dish of <a href="/wiki/Cabbage" title="Cabbage">cabbage</a>, though it is more typically eaten with <a href="/wiki/Collard_greens" class="mw-redirect" title="Collard greens">collard greens</a> (<i>sukuma wiki</i>).</div></div></div><div class="thumb tright"><div class="thumbinner" style="width:222px;"><a href="/wiki/File:Forodhani_park_food_stand.jpg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Forodhani_park_food_stand.jpg/220px-Forodhani_park_food_stand.jpg" decoding="async" width="220" height="147" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Forodhani_park_food_stand.jpg/330px-Forodhani_park_food_stand.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Forodhani_park_food_stand.jpg/440px-Forodhani_park_food_stand.jpg 2x" data-file-width="1600" data-file-height="1067"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Forodhani_park_food_stand.jpg" class="internal" title="Enlarge"></a></div>Barbecued beef cubes and seafood in <a href="/wiki/Forodhani_Gardens" title="Forodhani Gardens">Forodhani Gardens</a>, <a href="/wiki/Zanzibar" title="Zanzibar">Zanzibar</a></div></div></div><div class="thumb tright"><div class="thumbinner" style="width:222px;"><a href="/wiki/File:Kitcha_fit_fit.png" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/b/b5/Kitcha_fit_fit.png/220px-Kitcha_fit_fit.png" decoding="async" width="220" height="231" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/b/b5/Kitcha_fit_fit.png/330px-Kitcha_fit_fit.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/b/b5/Kitcha_fit_fit.png/440px-Kitcha_fit_fit.png 2x" data-file-width="828" data-file-height="868"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Kitcha_fit_fit.png" class="internal" title="Enlarge"></a></div><i><a href="/wiki/Kitcha_fit-fit" class="mw-redirect" title="Kitcha fit-fit">Kitcha fit-fit</a></i>, a mainstay in <a href="/wiki/Eritrean_cuisine" title="Eritrean cuisine">Eritrean cuisine</a>, is presented with a scoop of fresh yoghurt and topped with berbere (spice).</div></div></div><p>The cuisine of <a href="/wiki/East_Africa" title="East Africa">East Africa</a> varies from area to area. In the inland savannah, the traditional cuisine of cattle-keeping peoples is distinctive in that meat products are generally absent. <a href="/wiki/Cattle" title="Cattle">Cattle</a>, <a href="/wiki/Sheep" title="Sheep">sheep</a>, <a href="/wiki/Pigs" class="mw-redirect" title="Pigs">pigs</a> and <a href="/wiki/Goat" title="Goat">goats</a> were regarded as a form of <a href="/wiki/Currency" title="Currency">currency</a><sup id="cite_ref-14" class="reference"><a href="#cite_note-14">[14]</a></sup> and a store of wealth, and are not generally consumed as food.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (October 2017)">citation needed</span></a></i>]</sup>
</p><p>In some areas, traditional <a href="/wiki/East_Africa" title="East Africa">East Africans</a> consume the milk and blood of cattle, but rarely the meat. Elsewhere, other peoples are farmers who grow a variety of grains and vegetables. <a href="/wiki/Maize" title="Maize">Maize</a> (corn) is the basis of <a href="/wiki/Ugali" title="Ugali">ugali</a>, the local version of West and Central Africa's <a href="/wiki/Fufu" title="Fufu">fufu</a>. Ugali is a starch dish eaten with meats or stews. In <a href="/wiki/Uganda" title="Uganda">Uganda</a>, steamed green <a href="/wiki/Banana" title="Banana">bananas</a> called <a href="/wiki/Matoke" class="mw-redirect" title="Matoke">matoke</a> provide the starch filler of many meals.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (October 2017)">citation needed</span></a></i>]</sup>
</p><p>Around 1000&nbsp;years ago, <a href="/wiki/Oman" title="Oman">Omani</a> and <a href="/wiki/Yemen" title="Yemen">Yemeni</a> merchants settled on the <a href="/wiki/Swahili_Coast" class="mw-redirect" title="Swahili Coast">Swahili Coast</a>. Middle Eastern influences are especially reflected in the <a href="/wiki/Swahili_culture" title="Swahili culture">Swahili</a> cuisine of the coast – steamed or cooked rice with spices in <a href="/wiki/Persian_people" class="mw-redirect" title="Persian people">Persian</a> style; <a href="/wiki/Saffron" title="Saffron">saffron</a>, <a href="/wiki/Clove" title="Clove">cloves</a>, <a href="/wiki/Cinnamon" title="Cinnamon">cinnamon</a> and several other spices; and <a href="/wiki/Pomegranate" title="Pomegranate">pomegranate</a> juice.<sup id="cite_ref-15" class="reference"><a href="#cite_note-15">[15]</a></sup>
</p><p>Several centuries later, the British and the Indians came, and both brought with them foods such as Indian spiced vegetable <a href="/wiki/Curries" class="mw-redirect" title="Curries">curries</a>, <a href="/wiki/Lentil" title="Lentil">lentil</a> <a href="/wiki/Soup" title="Soup">soups</a>, chapattis and a variety of <a href="/wiki/Pickling" title="Pickling">pickles</a> which have influenced various local dishes. Some common ingredients used in this region include oranges, lemons, limes, chillis, capsicum peppers, maize, tomatoes, and strawberries. 
</p><p>In the Horn of Africa, the main traditional dishes in <a href="/wiki/Eritrean_cuisine" title="Eritrean cuisine">Eritrean cuisine</a> and <a href="/wiki/Ethiopian_cuisine" title="Ethiopian cuisine">Ethiopian cuisine</a> are <i><a href="/wiki/Wat_(food)" title="Wat (food)">tsebhis</a></i> (stews) served with <i><a href="/wiki/Injera" title="Injera">injera</a></i><sup id="cite_ref-Neo-Africanism_2008_2-1" class="reference"><a href="#cite_note-Neo-Africanism_2008-2">[2]</a></sup><sup id="cite_ref-webcitation_16-0" class="reference"><a href="#cite_note-webcitation-16">[16]</a></sup> (flatbread made from <a href="/wiki/Teff" title="Teff">teff</a>,<sup id="cite_ref-webcitation_16-1" class="reference"><a href="#cite_note-webcitation-16">[16]</a></sup> <a href="/wiki/Wheat" title="Wheat">wheat</a>, or <a href="/wiki/Sorghum" title="Sorghum">sorghum</a>) and <a href="/w/index.php?title=Hilbet&amp;action=edit&amp;redlink=1" class="new" title="Hilbet (page does not exist)">hilbet</a> (paste made from <a href="/wiki/Legume" title="Legume">legumes</a>, mainly <a href="/wiki/Lentil" title="Lentil">lentils</a> and <a href="/wiki/Vicia_faba" title="Vicia faba">fava beans</a>). Eritrean and Ethiopian cuisine (especially in the northern half) are very similar, given the shared history of the two countries.
</p><p>Eritrean and Ethiopian food habits vary regionally. In the <a href="/wiki/Ethiopian_Highlands" title="Ethiopian Highlands">highlands</a>, injera is the staple diet and is eaten daily among the <a href="/wiki/Tigray-Tigrinya_people" class="mw-redirect" title="Tigray-Tigrinya people">Tigrinya</a>. Injera is made out of teff, wheat, barley, sorghum or corn, and resembles a spongy, slightly sour pancake. When eating, diners generally share food from a large tray placed in the center of a low dining table. Numerous injera is layered on this tray and topped with various spicy stews. Diners then break into the section of injera in front of them, tearing off pieces and dipping them into the stews.
</p><p>In the lowlands, the main dish is <i><a href="/wiki/Akelet" class="mw-redirect" title="Akelet">akelet</a></i>, a porridge-like dish made from wheat flour dough. A <a href="/wiki/Ladle_(spoon)" title="Ladle (spoon)">ladle</a> is used to scoop out the top, which is filled with berbere and butter sauce and surrounded by milk or yoghurt. A small piece of dough is broken and then used to scoop up the sauce.
</p><div class="thumb tleft"><div class="thumbinner" style="width:182px;"><a href="/wiki/File:Alicha_1.jpg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/0/03/Alicha_1.jpg/180px-Alicha_1.jpg" decoding="async" width="180" height="135" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/0/03/Alicha_1.jpg/270px-Alicha_1.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/0/03/Alicha_1.jpg/360px-Alicha_1.jpg 2x" data-file-width="800" data-file-height="600"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Alicha_1.jpg" class="internal" title="Enlarge"></a></div>Typical <a href="/wiki/Ethiopian_cuisine" title="Ethiopian cuisine">Ethiopian</a> and <a href="/wiki/Eritrean_cuisine" title="Eritrean cuisine">Eritrean cuisine</a>: <i><a href="/wiki/Injera" title="Injera">Injera</a></i> (pancake-like bread) and several kinds of <i>wat</i> (stew)</div></div></div><p>The best known Ethio-Eritrean cuisine consists of various vegetable or meat side dishes and entrées, usually a <i><a href="/wiki/Wat_(food)" title="Wat (food)">wat</a></i>, or thick <a href="/wiki/Stew" title="Stew">stew</a>, served atop <i>injera</i>, a large <a href="/wiki/Sourdough" title="Sourdough">sourdough</a> <a href="/wiki/Flatbread" title="Flatbread">flatbread</a> made of teff flour. One does not eat with utensils, but instead uses injera to scoop up the entrées and side dishes.
</p><p>Tihlo, prepared from roasted barley flour, is very popular in Amhara, Agame, and Awlaelo (Tigray). Traditional Ethiopian cuisine employs no <a href="/wiki/Pork" title="Pork">pork</a> or <a href="/wiki/Shellfish" title="Shellfish">shellfish</a> of any kind, as they are forbidden in the Jewish and <a href="/wiki/Ethiopian_Orthodox_Tewahedo_Church" title="Ethiopian Orthodox Tewahedo Church">Ethiopian Orthodox Christian</a> faiths. It is also very common to eat from the same dish in the center of the table with a group of people.
</p><div class="thumb tright"><div class="thumbinner" style="width:222px;"><a href="/wiki/File:Halwo_003.jpg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/d/df/Halwo_003.jpg/220px-Halwo_003.jpg" decoding="async" width="220" height="165" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/d/df/Halwo_003.jpg/330px-Halwo_003.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/d/df/Halwo_003.jpg/440px-Halwo_003.jpg 2x" data-file-width="1632" data-file-height="1224"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Halwo_003.jpg" class="internal" title="Enlarge"></a></div><i>Xalwo</i>, the <a href="/wiki/Somali_people" class="mw-redirect" title="Somali people">Somali</a> version of <a href="/wiki/Halva" title="Halva">halva</a>, is a festive dish in <a href="/wiki/Somali_cuisine" title="Somali cuisine">Somali cuisine</a>.</div></div></div><p><a href="/wiki/Somali_cuisine" title="Somali cuisine">Somali cuisine</a> varies from region to region and consists of an exotic <a href="/wiki/Fusion_cuisine" title="Fusion cuisine">mixture</a> of diverse culinary influences. It is the product of <a href="/wiki/Somalia" title="Somalia">Somalia</a>'s rich <a href="/wiki/Maritime_history_of_Somalia" title="Maritime history of Somalia">tradition of trade and commerce</a>. Despite the variety, there remains one thing that unites the various regional cuisines: all food is served <a href="/wiki/Halal" title="Halal">halal</a>. There are therefore no pork dishes, <a href="/wiki/Alcoholic_beverage" class="mw-redirect" title="Alcoholic beverage">alcohol</a> is not served, nothing that died on its own is eaten, and no blood is incorporated. <i>Qaddo</i> or lunch is often elaborate.
</p><p>Varieties of <i>bariis</i> (rice), the most popular probably being <a href="/wiki/Basmati" title="Basmati">basmati</a>, usually serve as the main dish. Spices like <a href="/wiki/Cumin" title="Cumin">cumin</a>, <a href="/wiki/Cardamom" title="Cardamom">cardamom</a>, <a href="/wiki/Cloves" class="mw-redirect" title="Cloves">cloves</a>, <a href="/wiki/Cinnamon" title="Cinnamon">cinnamon</a> and <a href="/wiki/Common_sage" class="mw-redirect" title="Common sage">sage</a> are used to aromatize these different rice dishes. Somalis serve dinner as late as 9 pm. During <a href="/wiki/Ramadan_(calendar_month)" title="Ramadan (calendar month)">Ramadan</a>, dinner is often served after <a href="/wiki/Tarawih" title="Tarawih">Tarawih</a> prayers, sometimes as late as 11 pm.
</p><p><i>Xalwo</i> (halwo) or <a href="/wiki/Halva" title="Halva">halva</a> is a popular confection served during special occasions such as <a href="/wiki/Eid_ul-Fitr" class="mw-redirect" title="Eid ul-Fitr">Eid</a> celebrations or wedding receptions. It is made from sugar, <a href="/wiki/Cornstarch" class="mw-redirect" title="Cornstarch">cornstarch</a>, <a href="/wiki/Cardamom" title="Cardamom">cardamom</a> powder, <a href="/wiki/Nutmeg" title="Nutmeg">nutmeg</a> powder, and <a href="/wiki/Ghee" title="Ghee">ghee</a>. Peanuts are also sometimes added to enhance texture and flavor.<sup id="cite_ref-17" class="reference"><a href="#cite_note-17">[17]</a></sup> After meals, homes are traditionally perfumed using <a href="/wiki/Frankincense" title="Frankincense">frankincense</a> (<i>lubaan</i>) or <a href="/wiki/Incense" title="Incense">incense</a> (<i>cuunsi</i>), which is prepared inside an <a href="/wiki/Censer" title="Censer">incense burner</a> referred to as a <i><a href="/wiki/Dabqaad" title="Dabqaad">dabqaad</a></i>.
</p>'''

east_africa = ["Djibouti", "Eritrea", "Ethiopia", "Somalia", "Sudan", "South Sudan", "Madagascar", "Mauritius", "Comoros", "Seychelles", "French", "Réunion", "Mayotte", "Uganda", "Rwanda", "Burundi", "Kenya", "Tanzania", "Mozambique", "Malawi", "Zambia", "Zimbabwe"]
regional_cuisine_df = food.get_links_from_raw_html(raw_html, "African_cuisine#East_Africa", "East Africa", dict_with_places=(regional_cuisine_df, east_africa))
regional_cuisine_df

In [ ]:
north_africa = ["Algeria", "Morocco", "Tunisia", "Libya", "Mauritania"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/Maghrebi_cuisine", "North Africa", dict_with_places=(regional_cuisine_df, north_africa))
regional_cuisine_df

In [ ]:
west_africa = ["Benin", "Burkina Faso", "Cape Verde", "Ivory Coast", "Gambia", "Ghana", "Guinea", "Guinea-Bissau", "Liberia", "Mali", "Mauritania", "Niger", "Nigeria", "Saint Helena", "Ascension", "Tristan da Cunha", "Senegal", "Sierra Leone", "Togo"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/West_African_cuisine", "West Africa", dict_with_places=(regional_cuisine_df, west_africa))
regional_cuisine_df

In [ ]:
raw_html = '''<div class="thumb tright"><div class="thumbinner" style="width:202px;"><a href="/wiki/File:Ugali_and_cabbage.jpg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/f/fe/Ugali_and_cabbage.jpg/200px-Ugali_and_cabbage.jpg" decoding="async" width="200" height="175" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/f/fe/Ugali_and_cabbage.jpg/300px-Ugali_and_cabbage.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/f/fe/Ugali_and_cabbage.jpg/400px-Ugali_and_cabbage.jpg 2x" data-file-width="2183" data-file-height="1912"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Ugali_and_cabbage.jpg" class="internal" title="Enlarge"></a></div><i><a href="/wiki/Ugali" title="Ugali">Ugali</a></i> is pictured here with a side dish of <a href="/wiki/Cabbage" title="Cabbage">cabbage</a>, though it is more typically eaten with <a href="/wiki/Collard_greens" class="mw-redirect" title="Collard greens">collard greens</a> (<i>sukuma wiki</i>).</div></div></div><div class="thumb tright"><div class="thumbinner" style="width:222px;"><a href="/wiki/File:Forodhani_park_food_stand.jpg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Forodhani_park_food_stand.jpg/220px-Forodhani_park_food_stand.jpg" decoding="async" width="220" height="147" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Forodhani_park_food_stand.jpg/330px-Forodhani_park_food_stand.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Forodhani_park_food_stand.jpg/440px-Forodhani_park_food_stand.jpg 2x" data-file-width="1600" data-file-height="1067"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Forodhani_park_food_stand.jpg" class="internal" title="Enlarge"></a></div>Barbecued beef cubes and seafood in <a href="/wiki/Forodhani_Gardens" title="Forodhani Gardens">Forodhani Gardens</a>, <a href="/wiki/Zanzibar" title="Zanzibar">Zanzibar</a></div></div></div><div class="thumb tright"><div class="thumbinner" style="width:222px;"><a href="/wiki/File:Kitcha_fit_fit.png" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/b/b5/Kitcha_fit_fit.png/220px-Kitcha_fit_fit.png" decoding="async" width="220" height="231" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/b/b5/Kitcha_fit_fit.png/330px-Kitcha_fit_fit.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/b/b5/Kitcha_fit_fit.png/440px-Kitcha_fit_fit.png 2x" data-file-width="828" data-file-height="868"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Kitcha_fit_fit.png" class="internal" title="Enlarge"></a></div><i><a href="/wiki/Kitcha_fit-fit" class="mw-redirect" title="Kitcha fit-fit">Kitcha fit-fit</a></i>, a mainstay in <a href="/wiki/Eritrean_cuisine" title="Eritrean cuisine">Eritrean cuisine</a>, is presented with a scoop of fresh yoghurt and topped with berbere (spice).</div></div></div><p>The cuisine of <a href="/wiki/East_Africa" title="East Africa">East Africa</a> varies from area to area. In the inland savannah, the traditional cuisine of cattle-keeping peoples is distinctive in that meat products are generally absent. <a href="/wiki/Cattle" title="Cattle">Cattle</a>, <a href="/wiki/Sheep" title="Sheep">sheep</a>, <a href="/wiki/Pigs" class="mw-redirect" title="Pigs">pigs</a> and <a href="/wiki/Goat" title="Goat">goats</a> were regarded as a form of <a href="/wiki/Currency" title="Currency">currency</a><sup id="cite_ref-14" class="reference"><a href="#cite_note-14">[14]</a></sup> and a store of wealth, and are not generally consumed as food.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (October 2017)">citation needed</span></a></i>]</sup>
</p><p>In some areas, traditional <a href="/wiki/East_Africa" title="East Africa">East Africans</a> consume the milk and blood of cattle, but rarely the meat. Elsewhere, other peoples are farmers who grow a variety of grains and vegetables. <a href="/wiki/Maize" title="Maize">Maize</a> (corn) is the basis of <a href="/wiki/Ugali" title="Ugali">ugali</a>, the local version of West and Central Africa's <a href="/wiki/Fufu" title="Fufu">fufu</a>. Ugali is a starch dish eaten with meats or stews. In <a href="/wiki/Uganda" title="Uganda">Uganda</a>, steamed green <a href="/wiki/Banana" title="Banana">bananas</a> called <a href="/wiki/Matoke" class="mw-redirect" title="Matoke">matoke</a> provide the starch filler of many meals.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (October 2017)">citation needed</span></a></i>]</sup>
</p><p>Around 1000&nbsp;years ago, <a href="/wiki/Oman" title="Oman">Omani</a> and <a href="/wiki/Yemen" title="Yemen">Yemeni</a> merchants settled on the <a href="/wiki/Swahili_Coast" class="mw-redirect" title="Swahili Coast">Swahili Coast</a>. Middle Eastern influences are especially reflected in the <a href="/wiki/Swahili_culture" title="Swahili culture">Swahili</a> cuisine of the coast – steamed or cooked rice with spices in <a href="/wiki/Persian_people" class="mw-redirect" title="Persian people">Persian</a> style; <a href="/wiki/Saffron" title="Saffron">saffron</a>, <a href="/wiki/Clove" title="Clove">cloves</a>, <a href="/wiki/Cinnamon" title="Cinnamon">cinnamon</a> and several other spices; and <a href="/wiki/Pomegranate" title="Pomegranate">pomegranate</a> juice.<sup id="cite_ref-15" class="reference"><a href="#cite_note-15">[15]</a></sup>
</p><p>Several centuries later, the British and the Indians came, and both brought with them foods such as Indian spiced vegetable <a href="/wiki/Curries" class="mw-redirect" title="Curries">curries</a>, <a href="/wiki/Lentil" title="Lentil">lentil</a> <a href="/wiki/Soup" title="Soup">soups</a>, chapattis and a variety of <a href="/wiki/Pickling" title="Pickling">pickles</a> which have influenced various local dishes. Some common ingredients used in this region include oranges, lemons, limes, chillis, capsicum peppers, maize, tomatoes, and strawberries. 
</p><p>In the Horn of Africa, the main traditional dishes in <a href="/wiki/Eritrean_cuisine" title="Eritrean cuisine">Eritrean cuisine</a> and <a href="/wiki/Ethiopian_cuisine" title="Ethiopian cuisine">Ethiopian cuisine</a> are <i><a href="/wiki/Wat_(food)" title="Wat (food)">tsebhis</a></i> (stews) served with <i><a href="/wiki/Injera" title="Injera">injera</a></i><sup id="cite_ref-Neo-Africanism_2008_2-1" class="reference"><a href="#cite_note-Neo-Africanism_2008-2">[2]</a></sup><sup id="cite_ref-webcitation_16-0" class="reference"><a href="#cite_note-webcitation-16">[16]</a></sup> (flatbread made from <a href="/wiki/Teff" title="Teff">teff</a>,<sup id="cite_ref-webcitation_16-1" class="reference"><a href="#cite_note-webcitation-16">[16]</a></sup> <a href="/wiki/Wheat" title="Wheat">wheat</a>, or <a href="/wiki/Sorghum" title="Sorghum">sorghum</a>) and <a href="/w/index.php?title=Hilbet&amp;action=edit&amp;redlink=1" class="new" title="Hilbet (page does not exist)">hilbet</a> (paste made from <a href="/wiki/Legume" title="Legume">legumes</a>, mainly <a href="/wiki/Lentil" title="Lentil">lentils</a> and <a href="/wiki/Vicia_faba" title="Vicia faba">fava beans</a>). Eritrean and Ethiopian cuisine (especially in the northern half) are very similar, given the shared history of the two countries.
</p><p>Eritrean and Ethiopian food habits vary regionally. In the <a href="/wiki/Ethiopian_Highlands" title="Ethiopian Highlands">highlands</a>, injera is the staple diet and is eaten daily among the <a href="/wiki/Tigray-Tigrinya_people" class="mw-redirect" title="Tigray-Tigrinya people">Tigrinya</a>. Injera is made out of teff, wheat, barley, sorghum or corn, and resembles a spongy, slightly sour pancake. When eating, diners generally share food from a large tray placed in the center of a low dining table. Numerous injera is layered on this tray and topped with various spicy stews. Diners then break into the section of injera in front of them, tearing off pieces and dipping them into the stews.
</p><p>In the lowlands, the main dish is <i><a href="/wiki/Akelet" class="mw-redirect" title="Akelet">akelet</a></i>, a porridge-like dish made from wheat flour dough. A <a href="/wiki/Ladle_(spoon)" title="Ladle (spoon)">ladle</a> is used to scoop out the top, which is filled with berbere and butter sauce and surrounded by milk or yoghurt. A small piece of dough is broken and then used to scoop up the sauce.
</p><div class="thumb tleft"><div class="thumbinner" style="width:182px;"><a href="/wiki/File:Alicha_1.jpg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/0/03/Alicha_1.jpg/180px-Alicha_1.jpg" decoding="async" width="180" height="135" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/0/03/Alicha_1.jpg/270px-Alicha_1.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/0/03/Alicha_1.jpg/360px-Alicha_1.jpg 2x" data-file-width="800" data-file-height="600"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Alicha_1.jpg" class="internal" title="Enlarge"></a></div>Typical <a href="/wiki/Ethiopian_cuisine" title="Ethiopian cuisine">Ethiopian</a> and <a href="/wiki/Eritrean_cuisine" title="Eritrean cuisine">Eritrean cuisine</a>: <i><a href="/wiki/Injera" title="Injera">Injera</a></i> (pancake-like bread) and several kinds of <i>wat</i> (stew)</div></div></div><p>The best known Ethio-Eritrean cuisine consists of various vegetable or meat side dishes and entrées, usually a <i><a href="/wiki/Wat_(food)" title="Wat (food)">wat</a></i>, or thick <a href="/wiki/Stew" title="Stew">stew</a>, served atop <i>injera</i>, a large <a href="/wiki/Sourdough" title="Sourdough">sourdough</a> <a href="/wiki/Flatbread" title="Flatbread">flatbread</a> made of teff flour. One does not eat with utensils, but instead uses injera to scoop up the entrées and side dishes.
</p><p>Tihlo, prepared from roasted barley flour, is very popular in Amhara, Agame, and Awlaelo (Tigray). Traditional Ethiopian cuisine employs no <a href="/wiki/Pork" title="Pork">pork</a> or <a href="/wiki/Shellfish" title="Shellfish">shellfish</a> of any kind, as they are forbidden in the Jewish and <a href="/wiki/Ethiopian_Orthodox_Tewahedo_Church" title="Ethiopian Orthodox Tewahedo Church">Ethiopian Orthodox Christian</a> faiths. It is also very common to eat from the same dish in the center of the table with a group of people.
</p><div class="thumb tright"><div class="thumbinner" style="width:222px;"><a href="/wiki/File:Halwo_003.jpg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/d/df/Halwo_003.jpg/220px-Halwo_003.jpg" decoding="async" width="220" height="165" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/d/df/Halwo_003.jpg/330px-Halwo_003.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/d/df/Halwo_003.jpg/440px-Halwo_003.jpg 2x" data-file-width="1632" data-file-height="1224"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Halwo_003.jpg" class="internal" title="Enlarge"></a></div><i>Xalwo</i>, the <a href="/wiki/Somali_people" class="mw-redirect" title="Somali people">Somali</a> version of <a href="/wiki/Halva" title="Halva">halva</a>, is a festive dish in <a href="/wiki/Somali_cuisine" title="Somali cuisine">Somali cuisine</a>.</div></div></div><p><a href="/wiki/Somali_cuisine" title="Somali cuisine">Somali cuisine</a> varies from region to region and consists of an exotic <a href="/wiki/Fusion_cuisine" title="Fusion cuisine">mixture</a> of diverse culinary influences. It is the product of <a href="/wiki/Somalia" title="Somalia">Somalia</a>'s rich <a href="/wiki/Maritime_history_of_Somalia" title="Maritime history of Somalia">tradition of trade and commerce</a>. Despite the variety, there remains one thing that unites the various regional cuisines: all food is served <a href="/wiki/Halal" title="Halal">halal</a>. There are therefore no pork dishes, <a href="/wiki/Alcoholic_beverage" class="mw-redirect" title="Alcoholic beverage">alcohol</a> is not served, nothing that died on its own is eaten, and no blood is incorporated. <i>Qaddo</i> or lunch is often elaborate.
</p><p>Varieties of <i>bariis</i> (rice), the most popular probably being <a href="/wiki/Basmati" title="Basmati">basmati</a>, usually serve as the main dish. Spices like <a href="/wiki/Cumin" title="Cumin">cumin</a>, <a href="/wiki/Cardamom" title="Cardamom">cardamom</a>, <a href="/wiki/Cloves" class="mw-redirect" title="Cloves">cloves</a>, <a href="/wiki/Cinnamon" title="Cinnamon">cinnamon</a> and <a href="/wiki/Common_sage" class="mw-redirect" title="Common sage">sage</a> are used to aromatize these different rice dishes. Somalis serve dinner as late as 9 pm. During <a href="/wiki/Ramadan_(calendar_month)" title="Ramadan (calendar month)">Ramadan</a>, dinner is often served after <a href="/wiki/Tarawih" title="Tarawih">Tarawih</a> prayers, sometimes as late as 11 pm.
</p><p><i>Xalwo</i> (halwo) or <a href="/wiki/Halva" title="Halva">halva</a> is a popular confection served during special occasions such as <a href="/wiki/Eid_ul-Fitr" class="mw-redirect" title="Eid ul-Fitr">Eid</a> celebrations or wedding receptions. It is made from sugar, <a href="/wiki/Cornstarch" class="mw-redirect" title="Cornstarch">cornstarch</a>, <a href="/wiki/Cardamom" title="Cardamom">cardamom</a> powder, <a href="/wiki/Nutmeg" title="Nutmeg">nutmeg</a> powder, and <a href="/wiki/Ghee" title="Ghee">ghee</a>. Peanuts are also sometimes added to enhance texture and flavor.<sup id="cite_ref-17" class="reference"><a href="#cite_note-17">[17]</a></sup> After meals, homes are traditionally perfumed using <a href="/wiki/Frankincense" title="Frankincense">frankincense</a> (<i>lubaan</i>) or <a href="/wiki/Incense" title="Incense">incense</a> (<i>cuunsi</i>), which is prepared inside an <a href="/wiki/Censer" title="Censer">incense burner</a> referred to as a <i><a href="/wiki/Dabqaad" title="Dabqaad">dabqaad</a></i>.
</p>'''

southern_africa = ["Botswana", "Eswatini", "Lesotho", "Namibia", "South Africa"]
regional_cuisine_df = food.get_links_from_raw_html(raw_html, "African_cuisine#Southern_Africa", "Southern Africa", dict_with_places=(regional_cuisine_df, southern_africa))
regional_cuisine_df

In [ ]:
# Included "Overview" from South American cuisine
raw_html = '''<p>The <a href="/wiki/Amazon_basin" title="Amazon basin">Amazon basin</a> of <a href="/wiki/South_America" title="South America">South America</a> provides a plethora of fresh fish and <a href="/wiki/Tropical_fruit" class="mw-redirect" title="Tropical fruit">tropical fruits</a>.<sup id="cite_ref-3" class="reference"><a href="#cite_note-3">[3]</a></sup> The <a href="/wiki/Pacific_Ocean" title="Pacific Ocean">Pacific Ocean</a> provides a large amount of <a href="/wiki/Seafood" title="Seafood">seafood</a>, such as <a href="/wiki/King_crab" title="King crab">king crab</a> (typically caught at the southern end of the continent), <a href="/wiki/Lobster" title="Lobster">lobster</a> (found in great quantities from the <a href="/wiki/Juan_Fern%C3%A1ndez_Islands" title="Juan Fernández Islands">Juan Fernández Islands</a>), and <a href="/wiki/Antarctic_krill" title="Antarctic krill">Antarctic krill</a>, which was recently discovered. <a href="/wiki/Tuna" title="Tuna">Tuna</a> and tropical fish are caught all around the continent but are notably found in abundance near <a href="/wiki/Easter_Island" title="Easter Island">Easter Island</a>. The many plains on this continent make it rich for growing foods like <a href="/wiki/Cereals" class="mw-redirect" title="Cereals">cereals</a>, <a href="/wiki/Potatoes" class="mw-redirect" title="Potatoes">potatoes</a> and <a href="/wiki/Quinoa" title="Quinoa">quinoa</a>. In the <a href="/wiki/Patagonia" title="Patagonia">Patagonia</a> region in the south of <a href="/wiki/Argentina" title="Argentina">Argentina</a>, many people raise <a href="/wiki/Domestic_sheep" class="mw-redirect" title="Domestic sheep">sheep</a> and <a href="/wiki/Cattle" title="Cattle">cattle</a>. In <a href="/wiki/Brazil" title="Brazil">Brazil</a>, a traditional dish is the <a href="/wiki/Feijoada" title="Feijoada">feijoada</a>, a <a href="/wiki/Stew" title="Stew">stew</a> consisting of <a href="/wiki/Common_beans" class="mw-redirect" title="Common beans">beans</a> with <a href="/wiki/Beef" title="Beef">beef</a> and <a href="/wiki/Pork" title="Pork">pork</a>.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (July 2011)">citation needed</span></a></i>]</sup> <a href="/wiki/Rice_and_beans" title="Rice and beans">Rice and beans</a> is also very common in Brazil.<sup id="cite_ref-4" class="reference"><a href="#cite_note-4">[4]</a></sup> <a href="/wiki/Peruvian_cuisine" title="Peruvian cuisine">Peruvian cuisine</a> is largely influenced by traditional <a href="/wiki/Incan" class="mw-redirect" title="Incan">Incan</a> culture (see <a href="/wiki/Inca_cuisine" title="Inca cuisine">Incan cuisine</a>). The consumption of <a href="/wiki/Carne_asada" title="Carne asada">carne asada</a> is widespread throughout much of the continent.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (January 2018)">citation needed</span></a></i>]</sup><sup id="cite_ref-7" class="reference"><a href="#cite_note-7">[a]</a></sup>  It is popular to grill in the open air as in <a href="/wiki/Asado" title="Asado">asado</a> or <a href="/wiki/Churrasco" title="Churrasco">churrasco</a>. <a href="/wiki/Shredded_beef" title="Shredded beef">Shredded beef</a> is common throughout South America and Latin America and is served with a variety of foods.<sup id="cite_ref-Gutierrez_2013_p._150_8-0" class="reference"><a href="#cite_note-Gutierrez_2013_p._150-8">[5]</a></sup>
</p><div class="thumb tright"><div class="thumbinner" style="width:222px;"><a href="/wiki/File:Pachamanca_Peru.JPG" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/9/9d/Pachamanca_Peru.JPG/220px-Pachamanca_Peru.JPG" decoding="async" width="220" height="165" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/9/9d/Pachamanca_Peru.JPG/330px-Pachamanca_Peru.JPG 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/9/9d/Pachamanca_Peru.JPG/440px-Pachamanca_Peru.JPG 2x" data-file-width="2048" data-file-height="1536"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Pachamanca_Peru.JPG" class="internal" title="Enlarge"></a></div><a href="/wiki/Pachamanca" title="Pachamanca">Pachamanca</a></div></div></div><div role="note" class="hatnote navigation-not-searchable">Main articles: <a href="/wiki/Argentine_cuisine" title="Argentine cuisine">Argentine cuisine</a>, <a href="/wiki/Bolivian_cuisine" title="Bolivian cuisine">Bolivian cuisine</a>, <a href="/wiki/Chilean_cuisine" title="Chilean cuisine">Chilean cuisine</a>, <a href="/wiki/Ecuadorian_cuisine" title="Ecuadorian cuisine">Ecuadorian cuisine</a>, and <a href="/wiki/Peruvian_cuisine" title="Peruvian cuisine">Peruvian cuisine</a></div><p>The food of the Andes is highly influenced by the <a href="/wiki/Indigenous_peoples" title="Indigenous peoples">indigenous peoples</a>. The principal foods continue to be corn, potatoes<sup id="cite_ref-Krögel_2010_p._20_9-0" class="reference"><a href="#cite_note-Krögel_2010_p._20-9">[6]</a></sup> and other <a href="/wiki/Tuber" title="Tuber">tubers</a>. The meats most characteristic of this zone are the llama (Peru) and the guinea pig (Chile, Bolivia, Peru, Colombia and the Argentine northwest). In areas where there is fresh water, trout is consumed. <i><a href="/wiki/Chupe_andino" title="Chupe andino">Chupe andino</a></i> refers to various <a href="/wiki/Stew" title="Stew">stews</a><sup id="cite_ref-10" class="reference"><a href="#cite_note-10">[7]</a></sup> and <a href="/wiki/Soup" title="Soup">soups</a> that are prepared in the Andes mountains region. One of the most important drinks is <a href="/wiki/Chicha" title="Chicha">chicha</a>.<sup id="cite_ref-11" class="reference"><a href="#cite_note-11">[8]</a></sup>  Important dishes include <a href="/wiki/Humitas" class="mw-redirect" title="Humitas">humitas</a>, <a href="/wiki/Locro" title="Locro">locro</a>, chanfaina, <a href="/wiki/Arepas" class="mw-redirect" title="Arepas">arepas</a>, quimbolitos (dessert tamales), and peppers. A famous dish from the Peruvian Andes is <a href="/wiki/Pachamanca" title="Pachamanca">pachamanca</a>.<sup id="cite_ref-12" class="reference"><a href="#cite_note-12">[9]</a></sup> From the mixture of German, native cuisine, and the <a href="/wiki/Chilo%C3%A9_Archipelago" title="Chiloé Archipelago">Chiloé Archipelago</a> in the southern Andes comes <a href="/wiki/Valdiviano" title="Valdiviano">valdiviano</a> and <a href="/wiki/Curanto" title="Curanto">curanto</a>.
</p><p>The wetter areas of Peru produce sugar cane, lemon, bananas, and oranges. <a href="/wiki/Chancaca" title="Chancaca">Chancaca</a> is popular as well as <a href="/wiki/Carbonara" title="Carbonara">carbonara</a>, <a href="/wiki/Sancocho" title="Sancocho">sancocho</a>, huevos quimbos, potato pie, and <a href="/wiki/Ch%27arki" class="mw-redirect" title="Ch'arki">ch'arki</a>.
</p>'''

andes_south_america = ["Argentina", "Bolivia", "Chile", "Ecuador", "Peruvian"]
regional_cuisine_df = food.get_links_from_raw_html(raw_html, "South_American_cuisine#Andes", "Andes", dict_with_places=(regional_cuisine_df, andes_south_america))
regional_cuisine_df

In [ ]:
# Included "Overview" from South American cuisine
raw_html = '''<p>The <a href="/wiki/Amazon_basin" title="Amazon basin">Amazon basin</a> of <a href="/wiki/South_America" title="South America">South America</a> provides a plethora of fresh fish and <a href="/wiki/Tropical_fruit" class="mw-redirect" title="Tropical fruit">tropical fruits</a>.<sup id="cite_ref-3" class="reference"><a href="#cite_note-3">[3]</a></sup> The <a href="/wiki/Pacific_Ocean" title="Pacific Ocean">Pacific Ocean</a> provides a large amount of <a href="/wiki/Seafood" title="Seafood">seafood</a>, such as <a href="/wiki/King_crab" title="King crab">king crab</a> (typically caught at the southern end of the continent), <a href="/wiki/Lobster" title="Lobster">lobster</a> (found in great quantities from the <a href="/wiki/Juan_Fern%C3%A1ndez_Islands" title="Juan Fernández Islands">Juan Fernández Islands</a>), and <a href="/wiki/Antarctic_krill" title="Antarctic krill">Antarctic krill</a>, which was recently discovered. <a href="/wiki/Tuna" title="Tuna">Tuna</a> and tropical fish are caught all around the continent but are notably found in abundance near <a href="/wiki/Easter_Island" title="Easter Island">Easter Island</a>. The many plains on this continent make it rich for growing foods like <a href="/wiki/Cereals" class="mw-redirect" title="Cereals">cereals</a>, <a href="/wiki/Potatoes" class="mw-redirect" title="Potatoes">potatoes</a> and <a href="/wiki/Quinoa" title="Quinoa">quinoa</a>. In the <a href="/wiki/Patagonia" title="Patagonia">Patagonia</a> region in the south of <a href="/wiki/Argentina" title="Argentina">Argentina</a>, many people raise <a href="/wiki/Domestic_sheep" class="mw-redirect" title="Domestic sheep">sheep</a> and <a href="/wiki/Cattle" title="Cattle">cattle</a>. In <a href="/wiki/Brazil" title="Brazil">Brazil</a>, a traditional dish is the <a href="/wiki/Feijoada" title="Feijoada">feijoada</a>, a <a href="/wiki/Stew" title="Stew">stew</a> consisting of <a href="/wiki/Common_beans" class="mw-redirect" title="Common beans">beans</a> with <a href="/wiki/Beef" title="Beef">beef</a> and <a href="/wiki/Pork" title="Pork">pork</a>.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (July 2011)">citation needed</span></a></i>]</sup> <a href="/wiki/Rice_and_beans" title="Rice and beans">Rice and beans</a> is also very common in Brazil.<sup id="cite_ref-4" class="reference"><a href="#cite_note-4">[4]</a></sup> <a href="/wiki/Peruvian_cuisine" title="Peruvian cuisine">Peruvian cuisine</a> is largely influenced by traditional <a href="/wiki/Incan" class="mw-redirect" title="Incan">Incan</a> culture (see <a href="/wiki/Inca_cuisine" title="Inca cuisine">Incan cuisine</a>). The consumption of <a href="/wiki/Carne_asada" title="Carne asada">carne asada</a> is widespread throughout much of the continent.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (January 2018)">citation needed</span></a></i>]</sup><sup id="cite_ref-7" class="reference"><a href="#cite_note-7">[a]</a></sup>  It is popular to grill in the open air as in <a href="/wiki/Asado" title="Asado">asado</a> or <a href="/wiki/Churrasco" title="Churrasco">churrasco</a>. <a href="/wiki/Shredded_beef" title="Shredded beef">Shredded beef</a> is common throughout South America and Latin America and is served with a variety of foods.<sup id="cite_ref-Gutierrez_2013_p._150_8-0" class="reference"><a href="#cite_note-Gutierrez_2013_p._150-8">[5]</a></sup><div role="note" class="hatnote navigation-not-searchable">See also: <a href="/wiki/Brazilian_cuisine" title="Brazilian cuisine">Brazilian cuisine</a>, <a href="/wiki/Colombian_cuisine" title="Colombian cuisine">Colombian cuisine</a>, <a href="/wiki/Paraguayan_cuisine" title="Paraguayan cuisine">Paraguayan cuisine</a>, and <a href="/wiki/Venezuelan_cuisine" title="Venezuelan cuisine">Venezuelan cuisine</a></div><p>The tropical region of the continent is divided into two distinct areas, the coastal areas of the Atlantic and the Pacific and the Amazon area each with its distinct cuisines. Much of the fruits that are considered to be exotic are common in the tropical forests and fields, such as guava, pineapple, papaya, mango, banana, and elderberry.
</p><p>The climate and geography also favour a great variety of crops: potatoes, sweet potatoes, <a href="/wiki/Cassava" title="Cassava">cassava</a>, complemented with meat and fish; grains, principally rice, corn, and wheat and beans.
</p><p>In the coastal area <a href="/wiki/Ceviche" title="Ceviche">ceviche</a>, <a href="/wiki/Tostones" title="Tostones">tostones</a> or patacón, <a href="/wiki/Arepa" title="Arepa">arepa</a>, <a href="/wiki/Chipa" title="Chipa">chipa</a>, <a href="/wiki/Sancocho" title="Sancocho">sancocho</a>, <a href="/wiki/Pabell%C3%B3n_criollo" title="Pabellón criollo">pabellón criollo</a>, <a href="/wiki/Bandeja_paisa" title="Bandeja paisa">bandeja paisa</a>, <a href="/wiki/Guatita" class="mw-redirect" title="Guatita">guatita</a>, and <a href="/wiki/Sopa_paraguaya" title="Sopa paraguaya">sopa paraguaya</a> are common dishes.
</p><p>In Brazilian <a href="/wiki/Feijoada" title="Feijoada">feijoada</a>,<sup id="cite_ref-13" class="reference"><a href="#cite_note-13">[10]</a></sup> arroz carreteiro (cart riders rice), <a href="/wiki/Coxinha" title="Coxinha">coxinha</a>,<sup id="cite_ref-14" class="reference"><a href="#cite_note-14">[11]</a></sup> and <a href="/wiki/Farofa" title="Farofa">farofa</a> are common; <a href="/wiki/Bahia" title="Bahia">Bahia</a>, a state in Brazil, has its own cuisine which has heavy African influences.
</p><p>The Amazon area is known for its utilization of native meats such as the <a href="/wiki/Capybara" title="Capybara">capybara</a>, turtles, <a href="/wiki/Peccary" title="Peccary">peccary</a>,  and <a href="/wiki/Paca" title="Paca">paca</a>. Common dishes are <a href="/wiki/Juane" title="Juane">juane</a>, <a href="/wiki/Tacacho" title="Tacacho">tacacho</a>, <a href="/wiki/Tacac%C3%A1" title="Tacacá">tacacá</a>. There are a wide variety of fruits native to the Amazon with which a great variety of drinks are prepared.
</p>'''

tropical_south_america = ["Brazil", "Colombia", "Paraguay", "Venezuela"]
regional_cuisine_df = food.get_links_from_raw_html(raw_html, "South_American_cuisine#Tropical", "Tropical South America", dict_with_places=(regional_cuisine_df, tropical_south_america))
regional_cuisine_df

In [ ]:
# Included "Overview" from South American cuisine
raw_html = '''<p>The <a href="/wiki/Amazon_basin" title="Amazon basin">Amazon basin</a> of <a href="/wiki/South_America" title="South America">South America</a> provides a plethora of fresh fish and <a href="/wiki/Tropical_fruit" class="mw-redirect" title="Tropical fruit">tropical fruits</a>.<sup id="cite_ref-3" class="reference"><a href="#cite_note-3">[3]</a></sup> The <a href="/wiki/Pacific_Ocean" title="Pacific Ocean">Pacific Ocean</a> provides a large amount of <a href="/wiki/Seafood" title="Seafood">seafood</a>, such as <a href="/wiki/King_crab" title="King crab">king crab</a> (typically caught at the southern end of the continent), <a href="/wiki/Lobster" title="Lobster">lobster</a> (found in great quantities from the <a href="/wiki/Juan_Fern%C3%A1ndez_Islands" title="Juan Fernández Islands">Juan Fernández Islands</a>), and <a href="/wiki/Antarctic_krill" title="Antarctic krill">Antarctic krill</a>, which was recently discovered. <a href="/wiki/Tuna" title="Tuna">Tuna</a> and tropical fish are caught all around the continent but are notably found in abundance near <a href="/wiki/Easter_Island" title="Easter Island">Easter Island</a>. The many plains on this continent make it rich for growing foods like <a href="/wiki/Cereals" class="mw-redirect" title="Cereals">cereals</a>, <a href="/wiki/Potatoes" class="mw-redirect" title="Potatoes">potatoes</a> and <a href="/wiki/Quinoa" title="Quinoa">quinoa</a>. In the <a href="/wiki/Patagonia" title="Patagonia">Patagonia</a> region in the south of <a href="/wiki/Argentina" title="Argentina">Argentina</a>, many people raise <a href="/wiki/Domestic_sheep" class="mw-redirect" title="Domestic sheep">sheep</a> and <a href="/wiki/Cattle" title="Cattle">cattle</a>. In <a href="/wiki/Brazil" title="Brazil">Brazil</a>, a traditional dish is the <a href="/wiki/Feijoada" title="Feijoada">feijoada</a>, a <a href="/wiki/Stew" title="Stew">stew</a> consisting of <a href="/wiki/Common_beans" class="mw-redirect" title="Common beans">beans</a> with <a href="/wiki/Beef" title="Beef">beef</a> and <a href="/wiki/Pork" title="Pork">pork</a>.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (July 2011)">citation needed</span></a></i>]</sup> <a href="/wiki/Rice_and_beans" title="Rice and beans">Rice and beans</a> is also very common in Brazil.<sup id="cite_ref-4" class="reference"><a href="#cite_note-4">[4]</a></sup> <a href="/wiki/Peruvian_cuisine" title="Peruvian cuisine">Peruvian cuisine</a> is largely influenced by traditional <a href="/wiki/Incan" class="mw-redirect" title="Incan">Incan</a> culture (see <a href="/wiki/Inca_cuisine" title="Inca cuisine">Incan cuisine</a>). The consumption of <a href="/wiki/Carne_asada" title="Carne asada">carne asada</a> is widespread throughout much of the continent.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (January 2018)">citation needed</span></a></i>]</sup><sup id="cite_ref-7" class="reference"><a href="#cite_note-7">[a]</a></sup>  It is popular to grill in the open air as in <a href="/wiki/Asado" title="Asado">asado</a> or <a href="/wiki/Churrasco" title="Churrasco">churrasco</a>. <a href="/wiki/Shredded_beef" title="Shredded beef">Shredded beef</a> is common throughout South America and Latin America and is served with a variety of foods.<sup id="cite_ref-Gutierrez_2013_p._150_8-0" class="reference"><a href="#cite_note-Gutierrez_2013_p._150-8">[5]</a></sup>
</p><div role="note" class="hatnote navigation-not-searchable">Main articles: <a href="/wiki/Argentine_cuisine" title="Argentine cuisine">Argentine cuisine</a> and <a href="/wiki/Uruguayan_cuisine" title="Uruguayan cuisine">Uruguayan cuisine</a></div><p>The <a href="/wiki/Pampas" title="Pampas">pampas</a> have the most Italian and German influences. In Argentina, they are the center of the three typical Argentinian dishes <a href="/wiki/Dulce_de_leche" title="Dulce de leche">dulce de leche</a>, <a href="/wiki/Asado" title="Asado">asado</a> (churrasco in Brazil), and <a href="/wiki/Milanesa" title="Milanesa">milanesa</a>. Argentina pizza is different from Italian pizza, being closer to <a href="/wiki/Calzone" title="Calzone">calzones</a>. Pasta and polenta are common in Argentina and on the Pampas generally. <a href="/wiki/Empanadas" class="mw-redirect" title="Empanadas">Empanadas</a> and <a href="/wiki/Chorip%C3%A1n" title="Choripán">choripán</a> are very popular fast food in Argentina and Uruguay. <a href="/wiki/Churros" class="mw-redirect" title="Churros">Churros</a>, <a href="/wiki/Ensa%C3%AFmada" title="Ensaïmada">ensaïmada</a>, <a href="/wiki/Alfajor" title="Alfajor">alfajor</a>, Spanish tortillas with potato, meatballs, <a href="/wiki/Sopa_de_mondongo" title="Sopa de mondongo">sopa de mondongo</a>, and <a href="/wiki/Puchero" title="Puchero">puchero</a> are Spanish derived Pampas cuisine. <a href="/wiki/Mate_(drink)" title="Mate (drink)">Mate</a> is also found on the Pampas.<sup id="cite_ref-Henty_2016_15-0" class="reference"><a href="#cite_note-Henty_2016-15">[12]</a></sup>
</p>'''

pampas_south_america = ["Argentina", "Uruguay"]
regional_cuisine_df = food.get_links_from_raw_html(raw_html, "South_American_cuisine#Pampas", "Pampas", dict_with_places=(regional_cuisine_df, pampas_south_america))
regional_cuisine_df

In [ ]:
raw_html = '''<p><b>Latin American cuisine</b> is the typical foods, beverages, and cooking styles common to many of the countries and cultures in <a href="/wiki/Latin_America" title="Latin America">Latin America</a>. Latin America is a highly diverse area of land whose nations have varying cuisines. Some items typical of Latin American cuisine include <a href="/wiki/Maize" title="Maize">maize</a>-based dishes <a href="/wiki/Arepa" title="Arepa">arepas</a>, <a href="/wiki/Pupusa" title="Pupusa">pupusas</a>, <a href="/wiki/Taco" title="Taco">tacos</a>, <a href="/wiki/Tamale" title="Tamale">tamales</a>, <a href="/wiki/Tortilla" title="Tortilla">tortillas</a> and various <a href="/wiki/Salsa_(sauce)" title="Salsa (sauce)">salsas</a> and other condiments (<a href="/wiki/Guacamole" title="Guacamole">guacamole</a>, <a href="/wiki/Pico_de_gallo" title="Pico de gallo">pico de gallo</a>, <a href="/wiki/Mole_(sauce)" class="mw-redirect" title="Mole (sauce)">mole</a>, <a href="/wiki/Chimichurri" title="Chimichurri">chimichurri</a>, <a href="/wiki/Chili_pepper" title="Chili pepper">chili</a>, <a href="/wiki/Aj%C3%AD_(sauce)" title="Ají (sauce)">aji</a>, <a href="/wiki/Pebre" title="Pebre">pebre</a>). <a href="/wiki/Sofrito" title="Sofrito">Sofrito</a>, a culinary term that originally referred to a specific combination of sautéed or braised aromatics, exists in Latin American cuisine. It refers to a sauce of tomatoes, roasted bell peppers, garlic, onions and herbs.
</p><p>Latin American beverages are just as distinct as their foods. Some of the beverages can even date back to the times of the Native Americans. Some popular beverages include <a href="/wiki/Caf%C3%A9" class="mw-redirect" title="Café">coffee</a>, <a href="/wiki/Mate_(beverage)" class="mw-redirect" title="Mate (beverage)">mate</a>, <a href="/wiki/Hibiscus_tea" title="Hibiscus tea">hibiscus tea</a>, <a href="/wiki/Horchata" title="Horchata">horchata</a>, <a href="/wiki/Chicha" title="Chicha">chicha</a>, <a href="/wiki/Atole" title="Atole">atole</a>, <a href="/wiki/Hot_chocolate" title="Hot chocolate">cacao</a> and <a href="/wiki/Agua_fresca" class="mw-redirect" title="Agua fresca">aguas frescas</a>.
</p><p>Desserts in Latin America are generally very sweet in taste. They include <a href="/wiki/Dulce_de_leche" title="Dulce de leche">dulce de leche</a>, <a href="/wiki/Alfajor" title="Alfajor">alfajor</a>, <a href="/wiki/Rice_pudding" title="Rice pudding">rice pudding</a>, <a href="/wiki/Tres_leches_cake" title="Tres leches cake">tres leches cake</a>, <a href="/wiki/Teja_(confectionery)" title="Teja (confectionery)">teja</a> and <a href="/wiki/Cr%C3%A8me_caramel" title="Crème caramel">flan</a>.
</p>'''

latin_america = ["Paraguay", "Mexico", "Colombia", "Ecuador", "Bolivia", "Peru", "Venezuela", "Argentina", "Panama", "Chile", "Costa Rica", "Brazil", "Dominican Republic", "Puerto Rico", "El Salvador", "Guatemala", "Nicaragua", "Honduras", "Uruguay", "Canary Islands"]
regional_cuisine_df = food.get_links_from_raw_html(raw_html, "Latin_American_cuisine", "Latin America", dict_with_places=(regional_cuisine_df, latin_america))
regional_cuisine_df

In [ ]:
caribbean = ["Antigua and Barbuda", "The Bahamas", "Barbados", "Aruba", "Bonaire", "Sint Eustatius", "Saba", "Curaçao", "Sint Maarten", "Collectivity of Saint Martin", "Guadeloupe", "Martinique", "Saint Barthélemy", "Dominican Republic", "Haiti", "Navassa Island", "Puerto Rico", "United States Virgin Islands", "British Virgin Islands", "Anguilla", "Cayman Islands", "Montserrat", "Turks and Caicos Islands", "Cuba", "Dominica", "Jamaica", "Grenada", "Archipelago of San Andrés, Providencia and Santa Catalina", "Federal Dependencies of Venezuela", "Nueva Esparta", "Trinidad and Tobago", "Saint Kitts and Nevis", "Saint Lucia", "Saint Vincent and the Grenadines"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/Caribbean_cuisine", "Caribbean", dict_with_places=(regional_cuisine_df, caribbean))
regional_cuisine_df

In [ ]:
caucacus = ["Azerbaijan", "Armenia", "Abkhazia", "Georgia", "South Ossetia"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/List_of_dishes_from_the_Caucasus", "Caucasus", dict_with_places=(regional_cuisine_df, caucacus))
regional_cuisine_df

In [ ]:
mediterranean = ["Algeria", "Libya", "Morocco", "Tunisia", "Egypt", "Cyprus", "Jordan", "Lebanon", "Israel", "Palestine", "Syria", "Turkey", "Greece", "Italy", "San Marino", "France", "Spain", "Portugal", "Croatia", "Albania", "Bosnia and Herzegovina", "Slovenia", "Montenegro", "North Macedonia", "Malta", "Monaco"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/Mediterranean_cuisine", "Mediterranean", dict_with_places=(regional_cuisine_df, mediterranean))
regional_cuisine_df

In [ ]:
raw_html = '''<p><b>Asian cuisine</b> includes several major regional cuisines: <a href="#Central_Asian_cuisine">Central Asian</a>, <a href="#East_Asian_cuisine">East Asian</a>, <a href="#North_Asia_cuisine">North Asian</a>, <a href="#South_Asian_cuisine">South Asian</a>, <a href="#Southeast_Asian_cuisine">Southeast Asian</a>, and <a href="#Western_Asian_cuisine_(Middle_East)">West Asian</a>. A cuisine is a characteristic style of <a href="/wiki/Cooking" title="Cooking">cooking</a> practices and traditions,<sup id="cite_ref-1" class="reference"><a href="#cite_note-1">[1]</a></sup> usually associated with a specific <a href="/wiki/Culture" title="Culture">culture</a>. Asia, being the largest and most populous <a href="/wiki/Continent" title="Continent">continent</a>, is home to many cultures, many of which have their own characteristic cuisine.
</p><p>Ingredients common to many cultures in the East and Southeast regions of the continent include rice, ginger, garlic, sesame seeds, chilies, dried onions, soy, and tofu. <a href="/wiki/Stir_frying" title="Stir frying">Stir frying</a>, steaming, and <a href="/wiki/Deep_frying" title="Deep frying">deep frying</a> are common cooking methods.
</p><p>While rice is common to most Asian cuisines, different varieties are popular in the various regions. <a href="/wiki/Basmati_rice" class="mw-redirect" title="Basmati rice">Basmati rice</a> is popular in the <a href="/wiki/Indian_subcontinent" title="Indian subcontinent">Indian subcontinent</a>, <a href="/wiki/Jasmine_rice" title="Jasmine rice">Jasmine rice</a> is often found across <a href="/wiki/Southeast_Asia" title="Southeast Asia">Southeast Asia</a>, while long-grain rice is popular in China and short-grain in Japan and Korea.<sup id="cite_ref-quaker_2-0" class="reference"><a href="#cite_note-quaker-2">[2]</a></sup>
</p><p><a href="/wiki/Curry" title="Curry">Curry</a> is a common dish in South Asia, Southeast Asia, and East Asia. Curry dishes have their origins in the Indian subcontinent. With present-day Northern India, Bangladesh, and Pakistan mainly using a <a href="/wiki/Yogurt" title="Yogurt">yogurt</a> base, while those in present-day Southern India, Sri Lanka, and Southeast Asia generally use coconut milk as their foundation.<sup id="cite_ref-kraft_3-0" class="reference"><a href="#cite_note-kraft-3">[3]</a></sup>
</p><p>East Asian cuisine includes <a href="/wiki/Chinese_cuisine" title="Chinese cuisine">Chinese</a>, <a href="/wiki/Japanese_cuisine" title="Japanese cuisine">Japanese</a>, <a href="/wiki/Korean_cuisine" title="Korean cuisine">Korean</a>, <a href="/wiki/Mongolian_cuisine" title="Mongolian cuisine">Mongolian</a>, <a href="/wiki/Singaporean_cuisine" title="Singaporean cuisine">Singaporean</a>, <a href="/wiki/Taiwanese_cuisine" title="Taiwanese cuisine">Taiwanese</a> and <a href="/wiki/Tibetan_cuisine" title="Tibetan cuisine">Tibetan</a> food. Considering this is the most populated region of the world, it has many regional cuisines (especially in China). Examples of staple foods include <a href="/wiki/Rice" title="Rice">rice</a>, <a href="/wiki/Noodle" title="Noodle">noodles</a>, <a href="/wiki/Mung_bean" title="Mung bean">mung beans</a>, <a href="/wiki/Soybean" title="Soybean">soybeans</a>, <a href="/wiki/Seafood" title="Seafood">seafood</a> (Japan has the highest per capita consumption of seafood), <a href="/wiki/Mutton" class="mw-redirect" title="Mutton">mutton</a> (Mongolia), <a href="/wiki/Bok_choy" title="Bok choy">bok choy</a> (Chinese cabbage), and <a href="/wiki/Tea" title="Tea">tea</a>. Cuisines in this region are similar to Southeast Asian cuisine due to cultural and ethnic mixing.
</p>'''

east_asia = ["China", "Hong Kong", "Macau", "Japan", "Mongolia", "North Korea", "South Korea", "Taiwan", "Tibet"]
regional_cuisine_df = food.get_links_from_raw_html(raw_html, "Asian_cuisine#East_Asian_cuisine", "East Asia", dict_with_places=(regional_cuisine_df, east_asia))
regional_cuisine_df

In [ ]:
raw_html = '''<p><b>Asian cuisine</b> includes several major regional cuisines: <a href="#Central_Asian_cuisine">Central Asian</a>, <a href="#East_Asian_cuisine">East Asian</a>, <a href="#North_Asia_cuisine">North Asian</a>, <a href="#South_Asian_cuisine">South Asian</a>, <a href="#Southeast_Asian_cuisine">Southeast Asian</a>, and <a href="#Western_Asian_cuisine_(Middle_East)">West Asian</a>. A cuisine is a characteristic style of <a href="/wiki/Cooking" title="Cooking">cooking</a> practices and traditions,<sup id="cite_ref-1" class="reference"><a href="#cite_note-1">[1]</a></sup> usually associated with a specific <a href="/wiki/Culture" title="Culture">culture</a>. Asia, being the largest and most populous <a href="/wiki/Continent" title="Continent">continent</a>, is home to many cultures, many of which have their own characteristic cuisine.
</p><p>Ingredients common to many cultures in the East and Southeast regions of the continent include rice, ginger, garlic, sesame seeds, chilies, dried onions, soy, and tofu. <a href="/wiki/Stir_frying" title="Stir frying">Stir frying</a>, steaming, and <a href="/wiki/Deep_frying" title="Deep frying">deep frying</a> are common cooking methods.
</p><p>While rice is common to most Asian cuisines, different varieties are popular in the various regions. <a href="/wiki/Basmati_rice" class="mw-redirect" title="Basmati rice">Basmati rice</a> is popular in the <a href="/wiki/Indian_subcontinent" title="Indian subcontinent">Indian subcontinent</a>, <a href="/wiki/Jasmine_rice" title="Jasmine rice">Jasmine rice</a> is often found across <a href="/wiki/Southeast_Asia" title="Southeast Asia">Southeast Asia</a>, while long-grain rice is popular in China and short-grain in Japan and Korea.<sup id="cite_ref-quaker_2-0" class="reference"><a href="#cite_note-quaker-2">[2]</a></sup>
</p><p><a href="/wiki/Curry" title="Curry">Curry</a> is a common dish in South Asia, Southeast Asia, and East Asia. Curry dishes have their origins in the Indian subcontinent. With present-day Northern India, Bangladesh, and Pakistan mainly using a <a href="/wiki/Yogurt" title="Yogurt">yogurt</a> base, while those in present-day Southern India, Sri Lanka, and Southeast Asia generally use coconut milk as their foundation.<sup id="cite_ref-kraft_3-0" class="reference"><a href="#cite_note-kraft-3">[3]</a></sup>
</p><div role="note" class="hatnote navigation-not-searchable">Main article: <a href="/wiki/Central_Asian_cuisine" class="mw-redirect" title="Central Asian cuisine">Central Asian cuisine</a></div><ul class="gallery mw-gallery-traditional">
		<li class="gallerybox" style="width: 235px"><div style="width: 235px">
			<div class="thumb" style="width: 230px;"><div style="margin:15px auto;"><a href="/wiki/File:Central_Asia_(orthographic_projection).svg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/8/88/Central_Asia_%28orthographic_projection%29.svg/200px-Central_Asia_%28orthographic_projection%29.svg.png" decoding="async" width="200" height="200" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/8/88/Central_Asia_%28orthographic_projection%29.svg/300px-Central_Asia_%28orthographic_projection%29.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/8/88/Central_Asia_%28orthographic_projection%29.svg/400px-Central_Asia_%28orthographic_projection%29.svg.png 2x" data-file-width="553" data-file-height="553"></a></div></div>
			<div class="gallerytext">
<p>Location of <a href="/wiki/Central_Asia" title="Central Asia">Central Asia</a>.
</p>
			</div>
		</div></li>
</ul><p>Most Central Asian nations have similar cuisines to each other as well as their neighbors, taking many features of the neighboring cuisines of Western and Eastern Asia, particularly <a href="/wiki/Mongolia" title="Mongolia">Mongolia</a>.  A dish known as "plov", or "osh", for example, is a widespread variation of <a href="/wiki/Pilaf" title="Pilaf">pilaf</a>. However, many of the same countries use <a href="/wiki/Horse_meat" title="Horse meat">horse meat</a> and <a href="/wiki/Mutton" class="mw-redirect" title="Mutton">mutton</a> as the most common meats, similar to <a href="/wiki/Beef" title="Beef">beef</a>. This is owing to <a href="/wiki/Mongolian_cuisine" title="Mongolian cuisine">Mongolian cuisine</a>. In <a href="/wiki/Kazakhstan" title="Kazakhstan">Kazakhstan</a> and <a href="/wiki/Kyrgyzstan" title="Kyrgyzstan">Kyrgyzstan</a>, the cuisine has evolved to meet the needs of a nomad lifestyle.
</p><p><a href="/wiki/Kumis" title="Kumis">Kumis</a> is a widespread drink among <a href="/wiki/Turkic_peoples" title="Turkic peoples">Turkic peoples</a>, especially in Central Asia.
</p><p>Central Asia is also noted for being the birthplace of <a href="/wiki/Yogurt" title="Yogurt">yogurt</a>. Like kumis, it is widespread among Turkic peoples.
</p>'''

central_asia = ["Kazakhstan", "Kyrgyzstan", "Tajikistan", "Turkmenistan", "Uzbekistan"]
regional_cuisine_df = food.get_links_from_raw_html(raw_html, "Asian_cuisines#Central_Asian_cuisine", "Central Asia", dict_with_places=(regional_cuisine_df, central_asia))
regional_cuisine_df

In [ ]:
raw_html = '''<p><b>Asian cuisine</b> includes several major regional cuisines: <a href="#Central_Asian_cuisine">Central Asian</a>, <a href="#East_Asian_cuisine">East Asian</a>, <a href="#North_Asia_cuisine">North Asian</a>, <a href="#South_Asian_cuisine">South Asian</a>, <a href="#Southeast_Asian_cuisine">Southeast Asian</a>, and <a href="#Western_Asian_cuisine_(Middle_East)">West Asian</a>. A cuisine is a characteristic style of <a href="/wiki/Cooking" title="Cooking">cooking</a> practices and traditions,<sup id="cite_ref-1" class="reference"><a href="#cite_note-1">[1]</a></sup> usually associated with a specific <a href="/wiki/Culture" title="Culture">culture</a>. Asia, being the largest and most populous <a href="/wiki/Continent" title="Continent">continent</a>, is home to many cultures, many of which have their own characteristic cuisine.
</p><p>Ingredients common to many cultures in the East and Southeast regions of the continent include rice, ginger, garlic, sesame seeds, chilies, dried onions, soy, and tofu. <a href="/wiki/Stir_frying" title="Stir frying">Stir frying</a>, steaming, and <a href="/wiki/Deep_frying" title="Deep frying">deep frying</a> are common cooking methods.
</p><p>While rice is common to most Asian cuisines, different varieties are popular in the various regions. <a href="/wiki/Basmati_rice" class="mw-redirect" title="Basmati rice">Basmati rice</a> is popular in the <a href="/wiki/Indian_subcontinent" title="Indian subcontinent">Indian subcontinent</a>, <a href="/wiki/Jasmine_rice" title="Jasmine rice">Jasmine rice</a> is often found across <a href="/wiki/Southeast_Asia" title="Southeast Asia">Southeast Asia</a>, while long-grain rice is popular in China and short-grain in Japan and Korea.<sup id="cite_ref-quaker_2-0" class="reference"><a href="#cite_note-quaker-2">[2]</a></sup>
</p><p><a href="/wiki/Curry" title="Curry">Curry</a> is a common dish in South Asia, Southeast Asia, and East Asia. Curry dishes have their origins in the Indian subcontinent. With present-day Northern India, Bangladesh, and Pakistan mainly using a <a href="/wiki/Yogurt" title="Yogurt">yogurt</a> base, while those in present-day Southern India, Sri Lanka, and Southeast Asia generally use coconut milk as their foundation.<sup id="cite_ref-kraft_3-0" class="reference"><a href="#cite_note-kraft-3">[3]</a></sup>
</p><ul class="gallery mw-gallery-traditional">
		<li class="gallerybox" style="width: 235px"><div style="width: 235px">
			<div class="thumb" style="width: 230px;"><div style="margin:15px auto;"><a href="/wiki/File:North_Asia_(orthographic_projection).svg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/c/c9/North_Asia_%28orthographic_projection%29.svg/200px-North_Asia_%28orthographic_projection%29.svg.png" decoding="async" width="200" height="200" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/c/c9/North_Asia_%28orthographic_projection%29.svg/300px-North_Asia_%28orthographic_projection%29.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/c/c9/North_Asia_%28orthographic_projection%29.svg/400px-North_Asia_%28orthographic_projection%29.svg.png 2x" data-file-width="550" data-file-height="550"></a></div></div>
			<div class="gallerytext">
<p>Location of <a href="/wiki/Northern_Asia" class="mw-redirect" title="Northern Asia">Northern Asia</a>.
</p>
			</div>
		</div></li>
</ul><p>North Asian cuisine is often synonymous with <a href="/wiki/Russian_cuisine" title="Russian cuisine">Russian cuisine</a>, due to all of Northern Asia being a part of the Russian Federation.  However, some cultures or areas of Siberia have in-depth cuisine, such as the <a href="/wiki/Yakuts" title="Yakuts">Yakuts</a> (or Sakha) and <a href="/wiki/Yamal_cuisine" title="Yamal cuisine">Yamal cuisine</a>.  <a href="/wiki/Buryats" title="Buryats">Buryats</a> also have their own cuisine, although it is very similar to that of the related <a href="/wiki/Mongolian_cuisine" title="Mongolian cuisine">Mongolians</a>.
</p><p><a href="/wiki/Pelmeni" title="Pelmeni">Pelmeni</a>, originally a <a href="/wiki/Great_Perm" title="Great Perm">Permic</a> or <a href="/wiki/Yugra" title="Yugra">Ugric</a> dish, has entered into mainstream Russian cuisine as a well-known dish, but it can still be considered part of the Yamal cuisine for its area of origin.  Some speculate them to be a simplified version of the Chinese <a href="/wiki/Wonton" title="Wonton">wonton</a>.  In <a href="/wiki/Siberia" title="Siberia">Siberia</a>, pelmeni is frozen outdoors to preserve the meat inside throughout the long winter.  In <a href="/wiki/Yamal_Peninsula" title="Yamal Peninsula">Yamal</a>, other types of drying and preservation are common.  Key ingredients in most northern Siberian cuisine include fish and <a href="/wiki/Vaccinium_vitis-idaea" title="Vaccinium vitis-idaea">cowberries</a>, sometimes known as lingonberries in Europe and North America.  Yakuts, like many other <a href="/wiki/Turkic_peoples" title="Turkic peoples">Turkic-speaking peoples</a>, traditionally enjoy <a href="/wiki/Kumis" title="Kumis">kumis</a> as a common drink.
</p>'''

north_asia = ["Russia", "Mongolia"]
regional_cuisine_df = food.get_links_from_raw_html(raw_html, "Asian_cuisines#North_Asian_cuisine", "North Asia", dict_with_places=(regional_cuisine_df, north_asia))
regional_cuisine_df

In [ ]:
raw_html = '''<p><b>Asian cuisine</b> includes several major regional cuisines: <a href="#Central_Asian_cuisine">Central Asian</a>, <a href="#East_Asian_cuisine">East Asian</a>, <a href="#North_Asia_cuisine">North Asian</a>, <a href="#South_Asian_cuisine">South Asian</a>, <a href="#Southeast_Asian_cuisine">Southeast Asian</a>, and <a href="#Western_Asian_cuisine_(Middle_East)">West Asian</a>. A cuisine is a characteristic style of <a href="/wiki/Cooking" title="Cooking">cooking</a> practices and traditions,<sup id="cite_ref-1" class="reference"><a href="#cite_note-1">[1]</a></sup> usually associated with a specific <a href="/wiki/Culture" title="Culture">culture</a>. Asia, being the largest and most populous <a href="/wiki/Continent" title="Continent">continent</a>, is home to many cultures, many of which have their own characteristic cuisine.
</p><p>Ingredients common to many cultures in the East and Southeast regions of the continent include rice, ginger, garlic, sesame seeds, chilies, dried onions, soy, and tofu. <a href="/wiki/Stir_frying" title="Stir frying">Stir frying</a>, steaming, and <a href="/wiki/Deep_frying" title="Deep frying">deep frying</a> are common cooking methods.
</p><p>While rice is common to most Asian cuisines, different varieties are popular in the various regions. <a href="/wiki/Basmati_rice" class="mw-redirect" title="Basmati rice">Basmati rice</a> is popular in the <a href="/wiki/Indian_subcontinent" title="Indian subcontinent">Indian subcontinent</a>, <a href="/wiki/Jasmine_rice" title="Jasmine rice">Jasmine rice</a> is often found across <a href="/wiki/Southeast_Asia" title="Southeast Asia">Southeast Asia</a>, while long-grain rice is popular in China and short-grain in Japan and Korea.<sup id="cite_ref-quaker_2-0" class="reference"><a href="#cite_note-quaker-2">[2]</a></sup>
</p><p><a href="/wiki/Curry" title="Curry">Curry</a> is a common dish in South Asia, Southeast Asia, and East Asia. Curry dishes have their origins in the Indian subcontinent. With present-day Northern India, Bangladesh, and Pakistan mainly using a <a href="/wiki/Yogurt" title="Yogurt">yogurt</a> base, while those in present-day Southern India, Sri Lanka, and Southeast Asia generally use coconut milk as their foundation.<sup id="cite_ref-kraft_3-0" class="reference"><a href="#cite_note-kraft-3">[3]</a></sup>
</p><div role="note" class="hatnote navigation-not-searchable">Main article: <a href="/wiki/South_Asian_cuisine" class="mw-redirect" title="South Asian cuisine">South Asian cuisine</a></div><ul class="gallery mw-gallery-traditional">
		<li class="gallerybox" style="width: 235px"><div style="width: 235px">
			<div class="thumb" style="width: 230px;"><div style="margin:15px auto;"><a href="/wiki/File:South_Asia_(orthographic_projection)_without_national_boundaries.svg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/e/e3/South_Asia_%28orthographic_projection%29_without_national_boundaries.svg/200px-South_Asia_%28orthographic_projection%29_without_national_boundaries.svg.png" decoding="async" width="200" height="200" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/e/e3/South_Asia_%28orthographic_projection%29_without_national_boundaries.svg/300px-South_Asia_%28orthographic_projection%29_without_national_boundaries.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/e/e3/South_Asia_%28orthographic_projection%29_without_national_boundaries.svg/400px-South_Asia_%28orthographic_projection%29_without_national_boundaries.svg.png 2x" data-file-width="553" data-file-height="553"></a></div></div>
			<div class="gallerytext">
<p>Location of <a href="/wiki/South_Asia" title="South Asia">South Asia</a>.
</p>
			</div>
		</div></li>
</ul><p>South Asian cuisine includes the <a href="/wiki/Cuisine" title="Cuisine">cuisines</a> from the <a href="/wiki/Indian_subcontinent" title="Indian subcontinent">Indian subcontinent</a>. Foods in this area of the world are flavoured with various types of chili, black pepper, cloves, and other strong herbs and spices and often with flavoured butter and <a href="/wiki/Ghee" title="Ghee">ghee</a>. Turmeric and cumin are often used to make curries.
</p><p>Common meats include lamb, goat, fish, and chicken. <a href="/wiki/Beef" title="Beef">Beef</a> is less common than in <a href="/wiki/Western_world" title="Western world">Western</a> cuisines because cattle have a special place in <a href="/wiki/Hinduism" title="Hinduism">Hinduism</a>. Prohibitions against beef extend to the meat of cows, and <a href="/wiki/Yak" class="mw-redirect" title="Yak">yaks</a> to some extent.  <a href="/wiki/Pork" title="Pork">Pork</a> is considered a <a href="/wiki/Taboo_food_and_drink" class="mw-redirect" title="Taboo food and drink">taboo food item</a> by all Muslims and is avoided by some Hindus.
</p>'''

south_asia = ["Afghanistan", "Bangladesh", "Bhutan", "India", "Nepal", "Maldives", "Pakistan", "Sri Lanka"]
regional_cuisine_df = food.get_links_from_raw_html(raw_html, "Asian_cuisines#South_Asian_cuisine", "South Asia", dict_with_places=(regional_cuisine_df, south_asia))
regional_cuisine_df

In [ ]:
raw_html = '''<p><b>Asian cuisine</b> includes several major regional cuisines: <a href="#Central_Asian_cuisine">Central Asian</a>, <a href="#East_Asian_cuisine">East Asian</a>, <a href="#North_Asia_cuisine">North Asian</a>, <a href="#South_Asian_cuisine">South Asian</a>, <a href="#Southeast_Asian_cuisine">Southeast Asian</a>, and <a href="#Western_Asian_cuisine_(Middle_East)">West Asian</a>. A cuisine is a characteristic style of <a href="/wiki/Cooking" title="Cooking">cooking</a> practices and traditions,<sup id="cite_ref-1" class="reference"><a href="#cite_note-1">[1]</a></sup> usually associated with a specific <a href="/wiki/Culture" title="Culture">culture</a>. Asia, being the largest and most populous <a href="/wiki/Continent" title="Continent">continent</a>, is home to many cultures, many of which have their own characteristic cuisine.
</p><p>Ingredients common to many cultures in the East and Southeast regions of the continent include rice, ginger, garlic, sesame seeds, chilies, dried onions, soy, and tofu. <a href="/wiki/Stir_frying" title="Stir frying">Stir frying</a>, steaming, and <a href="/wiki/Deep_frying" title="Deep frying">deep frying</a> are common cooking methods.
</p><p>While rice is common to most Asian cuisines, different varieties are popular in the various regions. <a href="/wiki/Basmati_rice" class="mw-redirect" title="Basmati rice">Basmati rice</a> is popular in the <a href="/wiki/Indian_subcontinent" title="Indian subcontinent">Indian subcontinent</a>, <a href="/wiki/Jasmine_rice" title="Jasmine rice">Jasmine rice</a> is often found across <a href="/wiki/Southeast_Asia" title="Southeast Asia">Southeast Asia</a>, while long-grain rice is popular in China and short-grain in Japan and Korea.<sup id="cite_ref-quaker_2-0" class="reference"><a href="#cite_note-quaker-2">[2]</a></sup>
</p><p><a href="/wiki/Curry" title="Curry">Curry</a> is a common dish in South Asia, Southeast Asia, and East Asia. Curry dishes have their origins in the Indian subcontinent. With present-day Northern India, Bangladesh, and Pakistan mainly using a <a href="/wiki/Yogurt" title="Yogurt">yogurt</a> base, while those in present-day Southern India, Sri Lanka, and Southeast Asia generally use coconut milk as their foundation.<sup id="cite_ref-kraft_3-0" class="reference"><a href="#cite_note-kraft-3">[3]</a></sup>
</p><div role="note" class="hatnote navigation-not-searchable">Main article: <a href="/wiki/Southeast_Asian_cuisine" class="mw-redirect" title="Southeast Asian cuisine">Southeast Asian cuisine</a></div><ul class="gallery mw-gallery-traditional">
		<li class="gallerybox" style="width: 235px"><div style="width: 235px">
			<div class="thumb" style="width: 230px;"><div style="margin:15px auto;"><a href="/wiki/File:Southeast_Asia_(orthographic_projection).svg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/f/f7/Southeast_Asia_%28orthographic_projection%29.svg/200px-Southeast_Asia_%28orthographic_projection%29.svg.png" decoding="async" width="200" height="200" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/f/f7/Southeast_Asia_%28orthographic_projection%29.svg/299px-Southeast_Asia_%28orthographic_projection%29.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/f/f7/Southeast_Asia_%28orthographic_projection%29.svg/399px-Southeast_Asia_%28orthographic_projection%29.svg.png 2x" data-file-width="550" data-file-height="551"></a></div></div>
			<div class="gallerytext">
<p>Location of <a href="/wiki/Southeast_Asia" title="Southeast Asia">Southeast Asia</a>.
</p>
			</div>
		</div></li>
</ul><p>Southeast Asian cuisine includes a strong emphasis on lightly prepared dishes with a strong aromatic component that features such flavors as citrus and herbs such as <a href="/wiki/Kaffir_lime" title="Kaffir lime">lime</a>, <a href="/wiki/Coriander" title="Coriander">coriander</a>/<a href="/wiki/Cilantro" class="mw-redirect" title="Cilantro">cilantro</a> and <a href="/wiki/Basil" title="Basil">basil</a>. Ingredients in the region are similar with the ones in East Asian cuisines, using <a href="/wiki/Fish_sauce" title="Fish sauce">fish sauces</a> for sauce and the inclusion of ingredients such as <a href="/wiki/Galangal" title="Galangal">galangal</a>, <a href="/wiki/Tamarind" title="Tamarind">tamarind</a> and <a href="/wiki/Lemongrass" class="mw-redirect" title="Lemongrass">lemongrass</a>. Cooking methods also include a balance of <a href="/wiki/Stir-frying" class="mw-redirect" title="Stir-frying">stir-frying</a>, boiling and steaming.
</p>'''

southeast_asia = ["Andaman and Nicobar Islands", "Brunei", "Cambodia", "Christmas Island", "Cocos Islands", "East Timor", "Indonesia", "Laos", "Malaysia", "Myanmar", "Philippines", "Singapore", "Thailand", "Vietnam"]
regional_cuisine_df = food.get_links_from_raw_html(raw_html, "Asian_cuisines#Southeast_Asian_cuisine", "Southeast Asia", dict_with_places=(regional_cuisine_df, southeast_asia))
regional_cuisine_df

In [ ]:
raw_html = '''<p><b>Asian cuisine</b> includes several major regional cuisines: <a href="#Central_Asian_cuisine">Central Asian</a>, <a href="#East_Asian_cuisine">East Asian</a>, <a href="#North_Asia_cuisine">North Asian</a>, <a href="#South_Asian_cuisine">South Asian</a>, <a href="#Southeast_Asian_cuisine">Southeast Asian</a>, and <a href="#Western_Asian_cuisine_(Middle_East)">West Asian</a>. A cuisine is a characteristic style of <a href="/wiki/Cooking" title="Cooking">cooking</a> practices and traditions,<sup id="cite_ref-1" class="reference"><a href="#cite_note-1">[1]</a></sup> usually associated with a specific <a href="/wiki/Culture" title="Culture">culture</a>. Asia, being the largest and most populous <a href="/wiki/Continent" title="Continent">continent</a>, is home to many cultures, many of which have their own characteristic cuisine.
</p><p>Ingredients common to many cultures in the East and Southeast regions of the continent include rice, ginger, garlic, sesame seeds, chilies, dried onions, soy, and tofu. <a href="/wiki/Stir_frying" title="Stir frying">Stir frying</a>, steaming, and <a href="/wiki/Deep_frying" title="Deep frying">deep frying</a> are common cooking methods.
</p><p>While rice is common to most Asian cuisines, different varieties are popular in the various regions. <a href="/wiki/Basmati_rice" class="mw-redirect" title="Basmati rice">Basmati rice</a> is popular in the <a href="/wiki/Indian_subcontinent" title="Indian subcontinent">Indian subcontinent</a>, <a href="/wiki/Jasmine_rice" title="Jasmine rice">Jasmine rice</a> is often found across <a href="/wiki/Southeast_Asia" title="Southeast Asia">Southeast Asia</a>, while long-grain rice is popular in China and short-grain in Japan and Korea.<sup id="cite_ref-quaker_2-0" class="reference"><a href="#cite_note-quaker-2">[2]</a></sup>
</p><p><a href="/wiki/Curry" title="Curry">Curry</a> is a common dish in South Asia, Southeast Asia, and East Asia. Curry dishes have their origins in the Indian subcontinent. With present-day Northern India, Bangladesh, and Pakistan mainly using a <a href="/wiki/Yogurt" title="Yogurt">yogurt</a> base, while those in present-day Southern India, Sri Lanka, and Southeast Asia generally use coconut milk as their foundation.<sup id="cite_ref-kraft_3-0" class="reference"><a href="#cite_note-kraft-3">[3]</a></sup>
</p><div role="note" class="hatnote navigation-not-searchable">Main article: <a href="/wiki/West_Asian_cuisine" class="mw-redirect" title="West Asian cuisine">West Asian cuisine</a></div><ul class="gallery mw-gallery-traditional">
		<li class="gallerybox" style="width: 235px"><div style="width: 235px">
			<div class="thumb" style="width: 230px;"><div style="margin:15px auto;"><a href="/wiki/File:Western_Asia_(orthographic_projection).svg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/3/37/Western_Asia_%28orthographic_projection%29.svg/200px-Western_Asia_%28orthographic_projection%29.svg.png" decoding="async" width="200" height="200" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/3/37/Western_Asia_%28orthographic_projection%29.svg/300px-Western_Asia_%28orthographic_projection%29.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/3/37/Western_Asia_%28orthographic_projection%29.svg/400px-Western_Asia_%28orthographic_projection%29.svg.png 2x" data-file-width="683" data-file-height="683"></a></div></div>
			<div class="gallerytext">
<p>Location of <a href="/wiki/Western_Asia" title="Western Asia">Western Asia</a>.
</p>
			</div>
		</div></li>
</ul><p>West Asian cuisine significantly overlaps with the Middle Eastern cuisine and the inclusion of the <a href="/wiki/Caucasus" title="Caucasus">Caucasus</a>.
</p><p>West Asian cuisine is the <a href="/wiki/Cuisine" title="Cuisine">cuisine</a> of the various countries and peoples of West Asia. The cuisine of the region is diverse while having a degree of homogeneity.<sup id="cite_ref-globalgourmet_4-0" class="reference"><a href="#cite_note-globalgourmet-4">[4]</a></sup> Some commonly used ingredients include <a href="/wiki/Olive" title="Olive">olives</a> and <a href="/wiki/Olive_oil" title="Olive oil">olive oil</a>, <a href="/wiki/Pita" title="Pita">pitas</a>, <a href="/wiki/Honey" title="Honey">honey</a>, <a href="/wiki/Sesame" title="Sesame">sesame</a> seeds, <a href="/wiki/Phoenix_dactylifera" class="mw-redirect" title="Phoenix dactylifera">dates</a>,<sup id="cite_ref-globalgourmet_4-1" class="reference"><a href="#cite_note-globalgourmet-4">[4]</a></sup> <a href="/wiki/Sumac" title="Sumac">sumac</a>, <a href="/wiki/Chickpea" title="Chickpea">chickpeas</a>, <a href="/wiki/Mentha" title="Mentha">mint</a> and <a href="/wiki/Parsley" title="Parsley">parsley</a>. Some popular dishes include <a href="/wiki/Kibbeh" title="Kibbeh">kibbeh</a> and <a href="/wiki/Shawarma" title="Shawarma">shawarma</a>.
</p><p>Cereals constitute the basis of West Asian diet, both historically and today. <a href="/wiki/Wheat" title="Wheat">Wheat</a> and <a href="/wiki/Rice" title="Rice">rice</a> are the major and preferred sources of staple foods. <a href="/wiki/Barley" title="Barley">Barley</a> is also widely used in the region and <a href="/wiki/Maize" title="Maize">maize</a> has become common in some areas as well. Bread is a universal staple, eaten in one form or another by all classes and groups practically at every meal.
</p><p>Butter and clarified butter (also known as <a href="/wiki/Smen" title="Smen">samna/smen</a>) are, traditionally, the preferred medium of cooking. Olive oil is prevalent in the Mediterranean coastal areas. Christians use it during Lent, when meat and dairy products are excluded, and Jews use it in place of animal fats such as butter to avoid mixing meat and dairy products.
</p><p>Lamb and mutton have always been the favored meats of West Asia. Pork is prohibited in both Islam and Judaism, and as such is rarely eaten in the region. Prominent among the meat preparations are grilled meats, or <a href="/wiki/Kebab" title="Kebab">kebabs</a>. Meat and vegetable stews, served with rice, bulgur, or bread, are another form of meat preparation in the region.
</p><p>Vegetables and pulses are the predominant staples of the great majority of the people in the West Asia. They are boiled, stewed, grilled, stuffed, and cooked with meat and rice. Among the green leafy vegetables, many varieties of cabbage, spinach, and chard are widely used. Root and bulb vegetables, such as onions and garlic, as well as carrots, turnips, and beets are equally common.
</p>'''

west_asia = ["Turkey", "Bahrain", "Kuwait", "Oman", "Qatar", "Saudi Arabia", "United Arab Emirates", "Yemen", "Abkhazia", "Armenia", "Artsakh", "Azerbaijan", "Georgia", "South Ossetia", "Iraq", "Israel", "Jordan", "Lebanon", "Palestine", "Syria", "Iran", "Akrotiri and Dhekelia", "Cyprus", "Northern Cyprus", "Egypt"]
regional_cuisine_df = food.get_links_from_raw_html(raw_html, "Asian_cuisines#West_Asian_cuisine", "West Asia", dict_with_places=(regional_cuisine_df, west_asia))
regional_cuisine_df

In [ ]:
levantine = ["Assyrian", "Syria", "Iraq", "Lebanon", "Egypt", "Israel", "Jordan", "Turkey", "Cyprus", "Palestinian", "Kurdistan"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/Levantine_cuisine", "Levantine", dict_with_places=(regional_cuisine_df, levantine))
regional_cuisine_df

In [ ]:
assyrian = ["Syria", "Iraq", "Iran", "Turkey"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/Assyrian_cuisine", "Assyrian", dict_with_places=(regional_cuisine_df, assyrian))
regional_cuisine_df

In [ ]:
middle_east = ["Bahrain", "Cyprus", "Egypt", "Iran", "Iraq", "Israel", "Jordan", "Kuwait", "Lebanon", "Oman", "Palestine", "Qatar", "Saudi Arabia", "Syria", "Turkey", "United Arab Emirates", "Yemen", "Iran", "Israel", "Azerbaijan", "Armenia", "Kurdistan", "Cyprus"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/Middle_Eastern_cuisine", "Middle East", dict_with_places=(regional_cuisine_df, middle_east))
regional_cuisine_df

In [ ]:
indian_subcontinent = ["Bangladesh", "Bhutan", "India", "Maldives", "Nepal", "Pakistan", "Sri Lanka"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/Cuisine_of_the_Indian_subcontinent", "Indian subcontinent", dict_with_places=(regional_cuisine_df, indian_subcontinent))
regional_cuisine_df

In [ ]:
new_england = ["Maine", "Massachusetts", "New Hampshire", "Vermont", "Rhode Island", "Connecticut"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/Cuisine_of_New_England", "New England", dict_with_places=(regional_cuisine_df, new_england))
regional_cuisine_df

In [ ]:
mid_atlantic = ["New York", "New Jersey", "Delaware", "Pennsylvania", "Maryland"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/Cuisine_of_the_Mid-Atlantic_states", "Mid-Atlantic", dict_with_places=(regional_cuisine_df, mid_atlantic))
regional_cuisine_df

In [ ]:
midwest = ["Illinois", "Indiana", "Iowa", "Kansas", "Michigan", "Minnesota", "Missouri", "Nebraska", "North Dakota", "Ohio", "South Dakota", "Wisconsin"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/Cuisine_of_the_Midwestern_United_States", "Midwest", dict_with_places=(regional_cuisine_df, midwest))
regional_cuisine_df

In [ ]:
south = ["Alabama", "Arkansas", "Delaware", "Florida", "Georgia", "Kentucky", "Louisiana", "Maryland", "Mississippi", "North Carolina", "Oklahoma", "South Carolina", "Tennessee", "Texas", "Virginia", "West Virginia"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/Cuisine_of_the_Southern_United_States", "South", dict_with_places=(regional_cuisine_df, south))
regional_cuisine_df

In [ ]:
west = ["Alaska", "Arizona", "California", "Colorado", "Hawaii", "Idaho", "Montana", "Nevada", "New Mexico", "Oregon", "Utah", "Washington", "Wyoming"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/Cuisine_of_the_Western_United_States", "West", dict_with_places=(regional_cuisine_df, west))
regional_cuisine_df

In [ ]:
hawaii = ["Hawaii"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/Cuisine_of_Hawaii", "Hawaiian", dict_with_places=(regional_cuisine_df, hawaii))
regional_cuisine_df

In [ ]:
cajun = ["Louisiana"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/Cajun_cuisine", "Cajun", dict_with_places=(regional_cuisine_df, cajun))
regional_cuisine_df

In [ ]:
viennese = ["Vienna"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/Viennese_cuisine", "Vienna", dict_with_places=(regional_cuisine_df, viennese))
regional_cuisine_df

In [ ]:
baden = ["Baden"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/Baden_cuisine", "Baden", dict_with_places=(regional_cuisine_df, baden))
regional_cuisine_df

In [ ]:
bavaria = ["Bavaria"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/Bavarian_cuisine", "Bavaria", dict_with_places=(regional_cuisine_df, bavaria))
regional_cuisine_df

In [ ]:
lower_saxony = ["Lower Saxony"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/Lower_Saxon_cuisine", "Lower Saxon", dict_with_places=(regional_cuisine_df, lower_saxony))
regional_cuisine_df

In [ ]:
eastern_europe = ["Armenia", "Azerbaijan", "Belarus", "Bulgaria", "Georgia", "Kazakhstan", "Moldova", "Gagauzia", "Romania", "Russia"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/Eastern_European_cuisine", "Eastern Europe", dict_with_places=(regional_cuisine_df, eastern_europe))
regional_cuisine_df

In [ ]:
gibraltar = ["Gibraltar"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/Gibraltarian_cuisine", "Gibraltar", dict_with_places=(regional_cuisine_df, gibraltar))
regional_cuisine_df

In [ ]:
crete = ["Crete"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/Cretan_cuisine", "Crete", dict_with_places=(regional_cuisine_df, crete))
regional_cuisine_df

In [ ]:
catalonia = ["Catalonia"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/Catalan_cuisine", "Catalonia", dict_with_places=(regional_cuisine_df, catalonia))
regional_cuisine_df

In [ ]:
basque_country = ["Basque Country"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/Basque_cuisine", "Basque Country", dict_with_places=(regional_cuisine_df, basque_country))
regional_cuisine_df

In [ ]:
nordic = ["Sweden", "Norway", "Finland", "Iceland", "Denmark", "Faroe Islands", "Greenland", "Aland Islands", "Jan Mayen"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/New_Nordic_Cuisine", "Nordic countries", dict_with_places=(regional_cuisine_df, nordic))
regional_cuisine_df

In [ ]:
hessian = ["Hesse"]
regional_cuisine_df = food.get_cuisine_links("https://en.wikipedia.org/wiki/Hessian_cuisine", "Hessian", dict_with_places=(regional_cuisine_df, hessian))
regional_cuisine_df

## Pickling

In [ ]:
with open("regional_cuisine.pickle", "wb") as to_write:
    pickle.dump(regional_cuisine_df, to_write)